In [1]:
!pip install -U faiss-cpu chromadb tiktoken langchain langchain-community sentence-transformers unstructured transformers accelerate argparse numpy os re string sys faiss

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 2.7 MB/s eta 0:00:00
ERROR: Ignored the following versions that require a different python version: 1.21.2 Requires-Python >=3.7,<3.11; 1.21.3 Requires-Python >=3.7,<3.11; 1.21.4 Requires-Python >=3.7,<3.11; 1.21.5 Requires-Python >=3.7,<3.11; 1.21.6 Requires-Python >=3.7,<3.11
ERROR: Could not find a version that satisfies the requirement os (from versions: none)
ERROR: No matching distribution found for os


In [2]:
!pip install --upgrade --force-reinstall --no-cache-dir faiss-cpu


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 294.1 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.9/16.9 MB 298.2 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.5/66.5 kB 263.0 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 25.0
    Uninstalling packaging-25.0:
      Successfully uninstalled packaging-25.0
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.8.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
gensim 4.3.3 requires numpy<2.0,>=1.18.5, but you have numpy 2.3.2 which is inco

In [3]:
!pip uninstall -y transformers sentence_transformers scikit-learn scipy numpy

!pip install --no-cache-dir --upgrade numpy==1.26.4
!pip install --no-cache-dir --upgrade scipy==1.11.4
!pip install --no-cache-dir --upgrade scikit-learn==1.3.2
!pip install --no-cache-dir --upgrade transformers==4.37.2
!pip install --no-cache-dir --upgrade sentence-transformers==2.2.2
!pip install huggingface_hub==0.23.2

Found existing installation: transformers 4.52.4
Uninstalling transformers-4.52.4:
  Successfully uninstalled transformers-4.52.4
Found existing installation: sentence-transformers 4.1.0
Uninstalling sentence-transformers-4.1.0:
  Successfully uninstalled sentence-transformers-4.1.0
Found existing installation: scikit-learn 1.2.2
Uninstalling scikit-learn-1.2.2:
  Successfully uninstalled scikit-learn-1.2.2
Found existing installation: scipy 1.15.3
Uninstalling scipy-1.15.3:
  Successfully uninstalled scipy-1.15.3
Found existing installation: numpy 2.3.2
Uninstalling numpy-2.3.2:
  Successfully uninstalled numpy-2.3.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 172.9 MB/s eta 0:00:00a 0:00:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gensim 4.3.3 requires scipy<1.

In [ ]:
import os
os._exit(0)

修正评估模块后版本
但仍未解决问题

In [1]:
import json
import collections
import re
import string
import faiss
import numpy as np
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from sentence_transformers import SentenceTransformer
from tqdm import tqdm

# ------------------ SQuAD Evaluation Functions ------------------

def normalize_answer(s):
    if s is None:
        return ""
    def remove_articles(text):
        return re.sub(r'\b(a|an|the)\b', ' ', text)
    def white_space_fix(text):
        return ' '.join(text.split())
    def remove_punc(text):
        exclude = set(string.punctuation)
        return ''.join(ch for ch in text if ch not in exclude)
    def lower(text):
        return text.lower()
    return white_space_fix(remove_articles(remove_punc(lower(s))))

def get_tokens(s):
    if not s:
        return []
    return normalize_answer(s).split()

def compute_exact(a_gold, a_pred):
    return int(normalize_answer(a_gold) == normalize_answer(a_pred))

def compute_f1(a_gold, a_pred):
    gold_toks = get_tokens(a_gold)
    pred_toks = get_tokens(a_pred)
    common = collections.Counter(gold_toks) & collections.Counter(pred_toks)
    num_same = sum(common.values())
    if len(gold_toks) == 0 or len(pred_toks) == 0:
        return int(gold_toks == pred_toks)
    if num_same == 0:
        return 0
    precision = 1.0 * num_same / len(pred_toks)
    recall = 1.0 * num_same / len(gold_toks)
    return 2 * precision * recall / (precision + recall)

def get_raw_scores(dataset, preds):
    exact_scores = {}
    f1_scores = {}
    for article in dataset:
        for p in article['paragraphs']:
            for qa in p['qas']:
                qid = qa['id']
                gold_answers = [a['text'] for a in qa.get('answers', []) if normalize_answer(a.get('text', ''))]
                if not gold_answers:
                    gold_answers = ['']  # no answer questions
                a_pred = preds.get(qid, "")
                exact_scores[qid] = max(compute_exact(a, a_pred) for a in gold_answers)
                f1_scores[qid] = max(compute_f1(a, a_pred) for a in gold_answers)
    return exact_scores, f1_scores

def make_eval_dict(exact_scores, f1_scores):
    total = len(exact_scores)
    return collections.OrderedDict([
        ('exact', 100.0 * sum(exact_scores.values()) / total if total else 0),
        ('f1', 100.0 * sum(f1_scores.values()) / total if total else 0),
        ('total', total),
    ])

# ------------------ RAG 主体流程 ------------------

def load_squad_dataset(path):
    with open(path, 'r', encoding='utf-8') as f:
        data_json = json.load(f)
    return data_json['data']

def extract_contexts(dataset):
    contexts = []
    ctx_to_idx = {}
    for article in dataset:
        for p in article['paragraphs']:
            ctx = p['context']
            if ctx not in ctx_to_idx:
                ctx_to_idx[ctx] = len(contexts)
                contexts.append(ctx)
    return contexts, ctx_to_idx

def build_faiss_index(texts, embedder, batch_size=64):
    embs = []
    for i in range(0, len(texts), batch_size):
        batch = texts[i:i+batch_size]
        batch_emb = embedder.encode(batch, convert_to_numpy=True, normalize_embeddings=False, show_progress_bar=False)
        batch_emb = np.asarray(batch_emb, dtype=np.float32)
        embs.append(batch_emb)
    embeddings = np.vstack(embs)
    faiss.normalize_L2(embeddings)
    dim = embeddings.shape[1]
    index = faiss.IndexFlatIP(dim)
    index.add(embeddings)
    return index

def retrieve_top_k(question, index, embedder, contexts, k=15):
    q_emb = embedder.encode([question], convert_to_numpy=True, show_progress_bar=False)
    q_emb = np.asarray(q_emb, dtype=np.float32)
    faiss.normalize_L2(q_emb)
    D, I = index.search(q_emb, k)
    hits = []
    for idx in I[0]:
        if 0 <= idx < len(contexts):
            hits.append(contexts[idx])
    return hits

def chunk_and_truncate_contexts(contexts, max_chars=3000):
    out = []
    for c in contexts:
        if len(c) <= max_chars:
            out.append(c)
        else:
            out.append(c[:max_chars] + " ...")
    return out

def generate_answer(question, contexts, tokenizer, model, max_new_tokens=128):
    context_text = "\n".join(chunk_and_truncate_contexts(contexts))
    prompt = f"Question: {question}\nContext:\n{context_text}\nAnswer:"
    
    inputs = tokenizer(prompt, return_tensors="pt")
    device = next(model.parameters()).device
    inputs = {k: v.to(device) for k, v in inputs.items()}
    
    outputs = model.generate(
        **inputs,
        max_new_tokens=max_new_tokens,
        do_sample=False,
        num_beams=4,
        no_repeat_ngram_size=3,
        early_stopping=True
    )
    answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
    if answer.startswith(prompt):
        answer = answer[len(prompt):]
    return answer.strip()

def main():
    train_path = '/kaggle/input/squad-train/train-v2.0.json'
    dev_path = '/kaggle/input/squad-dev/dev-v2.0.json'

    sentence_transformer_name = 'all-mpnet-base-v2'
    model_name = "google-t5/t5-base"
    batch_size_embed = 64

    print("加载数据集...")
    train_data = load_squad_dataset(train_path)
    dev_data = load_squad_dataset(dev_path)

    print("提取并去重训练集上下文...")
    train_contexts, _ = extract_contexts(train_data)
    print(f"Unique training contexts: {len(train_contexts)}")

    print("加载句向量模型...")
    embedder = SentenceTransformer(sentence_transformer_name)

    print("建立FAISS索引...")
    index = build_faiss_index(train_contexts, embedder, batch_size=batch_size_embed)

    print(f"加载生成模型和tokenizer: {model_name}")
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = AutoModelForSeq2SeqLM.from_pretrained(model_name).to(device)
    model.eval()

    preds = {}

    print("开始对验证集生成回答...")
    count = 0
    max_test = 20
    for article in dev_data:
        for p in article['paragraphs']:
            for qa in p['qas']:
                if count >= max_test:
                    break
                qid = qa['id']
                question = qa['question']
                retrieved = retrieve_top_k(question, index, embedder, train_contexts, k=5)
                answer = generate_answer(question, retrieved, tokenizer, model)
                preds[qid] = answer
                print(f"[{count}] QID: {qid} 预测完成")
                count += 1
        if count >= max_test:
            break

    print("计算评测指标...")
    exact_scores, f1_scores = get_raw_scores(dev_data, preds)
    eval_results = make_eval_dict(exact_scores, f1_scores)

    print("评测结果：")
    for k, v in eval_results.items():
        print(f"{k}: {v:.2f}")

if __name__ == "__main__":
    main()


加载数据集...
提取并去重训练集上下文...
Unique training contexts: 19029
加载句向量模型...


.gitattributes: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

开始对验证集生成回答...
[0] QID: 56ddde6b9a695914005b9628 预测完成
[1] QID: 56ddde6b9a695914005b9629 预测完成
[2] QID: 56ddde6b9a695914005b962a 预测完成
[3] QID: 56ddde6b9a695914005b962b 预测完成
[4] QID: 56ddde6b9a695914005b962c 预测完成
[5] QID: 5ad39d53604f3c001a3fe8d1 预测完成
[6] QID: 5ad39d53604f3c001a3fe8d2 预测完成
[7] QID: 5ad39d53604f3c001a3fe8d3 预测完成
[8] QID: 5ad39d53604f3c001a3fe8d4 预测完成
[9] QID: 56dddf4066d3e219004dad5f 预测完成
[10] QID: 56dddf4066d3e219004dad60 预测完成
[11] QID: 56dddf4066d3e219004dad61 预测完成
[12] QID: 5ad3a266604f3c001a3fea27 预测完成
[13] QID: 5ad3a266604f3c001a3fea28 预测完成
[14] QID: 5ad3a266604f3c001a3fea29 预测完成
[15] QID: 5ad3a266604f3c001a3fea2a 预测完成
[16] QID: 5ad3a266604f3c001a3fea2b 预测完成
[17] QID: 56dde0379a695914005b9636 预测完成
[18] QID: 56dde0379a695914005b9637 预测完成
[19] QID: 5ad3ab70604f3c001a3feb89 预测完成
计算评测指标...
评测结果：
exact: 49.99
f1: 49.99
total: 11873.00


## 第一个可运行最小版本
使用google seq2seq模型，可能效果比opt好一点

In [1]:
import json
import collections
import re
import string
import faiss
import numpy as np
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from sentence_transformers import SentenceTransformer
from tqdm import tqdm

# ------------------ SQuAD Evaluation Functions ------------------

def normalize_answer(s):
    if s is None:
        return ""
    def remove_articles(text):
        return re.sub(r'\b(a|an|the)\b', ' ', text)
    def white_space_fix(text):
        return ' '.join(text.split())
    def remove_punc(text):
        exclude = set(string.punctuation)
        return ''.join(ch for ch in text if ch not in exclude)
    def lower(text):
        return text.lower()
    return white_space_fix(remove_articles(remove_punc(lower(s))))

def get_tokens(s):
    if not s:
        return []
    return normalize_answer(s).split()

def compute_exact(a_gold, a_pred):
    return int(normalize_answer(a_gold) == normalize_answer(a_pred))

def compute_f1(a_gold, a_pred):
    gold_toks = get_tokens(a_gold)
    pred_toks = get_tokens(a_pred)
    common = collections.Counter(gold_toks) & collections.Counter(pred_toks)
    num_same = sum(common.values())
    if len(gold_toks) == 0 or len(pred_toks) == 0:
        return int(gold_toks == pred_toks)
    if num_same == 0:
        return 0
    precision = 1.0 * num_same / len(pred_toks)
    recall = 1.0 * num_same / len(gold_toks)
    return 2 * precision * recall / (precision + recall)

def get_raw_scores(dataset, preds):
    exact_scores = {}
    f1_scores = {}
    for article in dataset:
        for p in article['paragraphs']:
            for qa in p['qas']:
                qid = qa['id']
                gold_answers = [a['text'] for a in qa.get('answers', []) if normalize_answer(a.get('text', ''))]
                if not gold_answers:
                    gold_answers = ['']  # no answer questions
                a_pred = preds.get(qid, "")
                exact_scores[qid] = max(compute_exact(a, a_pred) for a in gold_answers)
                f1_scores[qid] = max(compute_f1(a, a_pred) for a in gold_answers)
    return exact_scores, f1_scores

def make_eval_dict(exact_scores, f1_scores):
    total = len(exact_scores)
    return collections.OrderedDict([
        ('exact', 100.0 * sum(exact_scores.values()) / total if total else 0),
        ('f1', 100.0 * sum(f1_scores.values()) / total if total else 0),
        ('total', total),
    ])

# ------------------ RAG 主体流程 ------------------

def load_squad_dataset(path):
    with open(path, 'r', encoding='utf-8') as f:
        data_json = json.load(f)
    return data_json['data']

def extract_contexts(dataset):
    contexts = []
    ctx_to_idx = {}
    for article in dataset:
        for p in article['paragraphs']:
            ctx = p['context']
            if ctx not in ctx_to_idx:
                ctx_to_idx[ctx] = len(contexts)
                contexts.append(ctx)
    return contexts, ctx_to_idx

def build_faiss_index(texts, embedder, batch_size=256):
    embs = []
    for i in range(0, len(texts), batch_size):
        batch = texts[i:i+batch_size]
        batch_emb = embedder.encode(batch, convert_to_numpy=True, device='cuda', normalize_embeddings=True, show_progress_bar=False)
        batch_emb = np.asarray(batch_emb, dtype=np.float32)
        embs.append(batch_emb)
    embeddings = np.vstack(embs)
    # 因为 embedder.encode 已经 normalize_embeddings=True，这里不再调用 faiss.normalize_L2
    dim = embeddings.shape[1]
    index = faiss.IndexFlatIP(dim)
    index.add(embeddings)
    return index

def retrieve_top_k(question, index, embedder, contexts, k=5):
    q_emb = embedder.encode([question], convert_to_numpy=True, device='cuda', normalize_embeddings=True, show_progress_bar=False)
    q_emb = np.asarray(q_emb, dtype=np.float32)
    # 同理这里不调用 faiss.normalize_L2
    D, I = index.search(q_emb, k)
    hits = []
    for idx in I[0]:
        if 0 <= idx < len(contexts):
            hits.append(contexts[idx])
    return hits

def chunk_and_truncate_contexts(contexts, max_chars=3000):
    out = []
    for c in contexts:
        if len(c) <= max_chars:
            out.append(c)
        else:
            out.append(c[:max_chars] + " ...")
    return out

def generate_answer(question, contexts, tokenizer, model, max_new_tokens=64):
    context_text = "\n".join(chunk_and_truncate_contexts(contexts))
    prompt = f"Question: {question}\nContext:\n{context_text}\nAnswer (if no answer, respond with 'no answer'):"

    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=512)
    device = next(model.parameters()).device
    inputs = {k: v.to(device) for k, v in inputs.items()}

    outputs = model.generate(
        **inputs,
        max_new_tokens=max_new_tokens,
        do_sample=False,
        num_beams=4,
        no_repeat_ngram_size=3,
        early_stopping=True
    )
    answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
    if answer.startswith(prompt):
        answer = answer[len(prompt):]
    answer = answer.strip()
    if answer.lower() in ["false", "no answer", "none", ""]:
        answer = ""
    return answer



def main():
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    train_path = '/kaggle/input/squad-train/train-v2.0.json'
    dev_path = '/kaggle/input/squad-dev/dev-v2.0.json'

    sentence_transformer_name = 'all-mpnet-base-v2'
    model_name = "google/flan-t5-base"
    batch_size_embed = 256

    print("加载数据集...")
    train_data = load_squad_dataset(train_path)
    dev_data = load_squad_dataset(dev_path)

    print("提取并去重训练集上下文...")
    train_contexts, _ = extract_contexts(train_data)
    print(f"Unique training contexts: {len(train_contexts)}")

    print("加载句向量模型...")
    # 这里修改了device传参
    embedder = SentenceTransformer(sentence_transformer_name, device=device)

    print("建立FAISS索引...")
    index = build_faiss_index(train_contexts, embedder, batch_size=batch_size_embed)

    print(f"加载生成模型和tokenizer: {model_name}")
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSeq2SeqLM.from_pretrained(model_name).to(device)
    model.eval()

    preds = {}
    dev_subset = []  # 保存只用的前 20 个 QA

    print("开始对验证集生成回答...")
    count = 0
    max_test = 20
    for article in dev_data:
        for p in article['paragraphs']:
            for qa in p['qas']:
                if count >= max_test:
                    break
                qid = qa['id']
                question = qa['question']
                retrieved = retrieve_top_k(question, index, embedder, train_contexts, k=5)
                answer = generate_answer(question, retrieved, tokenizer, model)

                preds[qid] = answer
                dev_subset.append({
                    "id": qid,
                    "question": question,
                    "answers": qa["answers"],
                    "retrieved": retrieved,
                    "pred": answer
                })

                print(count)

                count += 1
            if count >= max_test:
                break
        if count >= max_test:
            break

    # 构造一个只包含前 20 个的 dev_data 子集
    filtered_dev_data = []
    for article in dev_data:
        new_article = {"paragraphs": []}
        for p in article['paragraphs']:
            new_p = {"qas": [], "context": p["context"]}
            for qa in p['qas']:
                if qa['id'] in preds:
                    new_p["qas"].append(qa)
            if new_p["qas"]:
                new_article["paragraphs"].append(new_p)
        if new_article["paragraphs"]:
            filtered_dev_data.append(new_article)

    print("计算评测指标...")
    exact_scores, f1_scores = get_raw_scores(filtered_dev_data, preds)
    eval_results = make_eval_dict(exact_scores, f1_scores)

    print("评测结果：")
    for k, v in eval_results.items():
        print(f"{k}: {v:.2f}")


if __name__ == "__main__":
    main()


加载数据集...
提取并去重训练集上下文...
Unique training contexts: 19029
加载句向量模型...
建立FAISS索引...
加载生成模型和tokenizer: google/flan-t5-base


/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

开始对验证集生成回答...
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
计算评测指标...
评测结果：
exact: 5.00
f1: 10.75
total: 20.00


# faiss加速后版本

In [4]:
import json
import collections
import re
import string
import faiss
import numpy as np
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from sentence_transformers import SentenceTransformer
from tqdm import tqdm

# ------------------ SQuAD Evaluation Functions ------------------

def normalize_answer(s):
    if s is None:
        return ""
    def remove_articles(text):
        return re.sub(r'\b(a|an|the)\b', ' ', text)
    def white_space_fix(text):
        return ' '.join(text.split())
    def remove_punc(text):
        exclude = set(string.punctuation)
        return ''.join(ch for ch in text if ch not in exclude)
    def lower(text):
        return text.lower()
    return white_space_fix(remove_articles(remove_punc(lower(s))))

def get_tokens(s):
    if not s:
        return []
    return normalize_answer(s).split()

def compute_exact(a_gold, a_pred):
    return int(normalize_answer(a_gold) == normalize_answer(a_pred))

def compute_f1(a_gold, a_pred):
    gold_toks = get_tokens(a_gold)
    pred_toks = get_tokens(a_pred)
    common = collections.Counter(gold_toks) & collections.Counter(pred_toks)
    num_same = sum(common.values())
    if len(gold_toks) == 0 or len(pred_toks) == 0:
        return int(gold_toks == pred_toks)
    if num_same == 0:
        return 0
    precision = 1.0 * num_same / len(pred_toks)
    recall = 1.0 * num_same / len(gold_toks)
    return 2 * precision * recall / (precision + recall)

def get_raw_scores(dataset, preds):
    exact_scores = {}
    f1_scores = {}
    for article in dataset:
        for p in article['paragraphs']:
            for qa in p['qas']:
                qid = qa['id']
                gold_answers = [a['text'] for a in qa.get('answers', []) if normalize_answer(a.get('text', ''))]
                if not gold_answers:
                    gold_answers = ['']  # no answer questions
                a_pred = preds.get(qid, "")
                exact_scores[qid] = max(compute_exact(a, a_pred) for a in gold_answers)
                f1_scores[qid] = max(compute_f1(a, a_pred) for a in gold_answers)
    return exact_scores, f1_scores

def make_eval_dict(exact_scores, f1_scores):
    total = len(exact_scores)
    return collections.OrderedDict([
        ('exact', 100.0 * sum(exact_scores.values()) / total if total else 0),
        ('f1', 100.0 * sum(f1_scores.values()) / total if total else 0),
        ('total', total),
    ])

# ------------------ RAG 主体流程 ------------------

def load_squad_dataset(path):
    with open(path, 'r', encoding='utf-8') as f:
        data_json = json.load(f)
    return data_json['data']

def extract_contexts(dataset):
    contexts = []
    ctx_to_idx = {}
    for article in dataset:
        for p in article['paragraphs']:
            ctx = p['context']
            if ctx not in ctx_to_idx:
                ctx_to_idx[ctx] = len(contexts)
                contexts.append(ctx)
    return contexts, ctx_to_idx

def build_faiss_index_ivfpq(texts, embedder, batch_size=256, nlist=100, m=16, nbits=8):
    # 编码所有文本
    embs = []
    for i in tqdm(range(0, len(texts), batch_size), desc="Embedding batches"):
        batch = texts[i:i+batch_size]
        batch_emb = embedder.encode(batch, convert_to_numpy=True, device='cuda', normalize_embeddings=True, show_progress_bar=False)
        batch_emb = np.asarray(batch_emb, dtype=np.float32)
        embs.append(batch_emb)
    embeddings = np.vstack(embs)

    dim = embeddings.shape[1]
    # IVF + PQ索引
    quantizer = faiss.IndexFlatIP(dim)  # 用内积量化器
    index = faiss.IndexIVFPQ(quantizer, dim, nlist, m, nbits)

    # 训练索引，必须先训练
    print("训练IVF-PQ索引...")
    index.train(embeddings)

    # 添加向量
    print("添加向量到索引...")
    index.add(embeddings)

    return index

def retrieve_top_k(question, index, embedder, contexts, k=5):
    q_emb = embedder.encode([question], convert_to_numpy=True, device='cuda', normalize_embeddings=True, show_progress_bar=False)
    q_emb = np.asarray(q_emb, dtype=np.float32)
    D, I = index.search(q_emb, k)
    hits = []
    for idx in I[0]:
        if 0 <= idx < len(contexts):
            hits.append(contexts[idx])
    return hits

def chunk_and_truncate_contexts(contexts, max_chars=3000):
    out = []
    for c in contexts:
        if len(c) <= max_chars:
            out.append(c)
        else:
            out.append(c[:max_chars] + " ...")
    return out

def generate_answer(question, contexts, tokenizer, model, max_new_tokens=64):
    context_text = "\n".join(chunk_and_truncate_contexts(contexts))
    prompt = f"Question: {question}\nContext:\n{context_text}\nAnswer (if no answer, respond with 'no answer'):"

    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=512)
    device = next(model.parameters()).device
    inputs = {k: v.to(device) for k, v in inputs.items()}

    outputs = model.generate(
        **inputs,
        max_new_tokens=max_new_tokens,
        do_sample=False,
        num_beams=4,
        no_repeat_ngram_size=3,
        early_stopping=True
    )
    answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
    if answer.startswith(prompt):
        answer = answer[len(prompt):]
    answer = answer.strip()
    if answer.lower() in ["false", "no answer", "none", ""]:
        answer = ""
    return answer

def main():
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    train_path = '/kaggle/input/squad-train/train-v2.0.json'
    dev_path = '/kaggle/input/squad-dev/dev-v2.0.json'

    sentence_transformer_name = 'all-mpnet-base-v2'
    model_name = "google/flan-t5-base"
    batch_size_embed = 256

    print("加载数据集...")
    train_data = load_squad_dataset(train_path)
    dev_data = load_squad_dataset(dev_path)

    print("提取并去重训练集上下文...")
    train_contexts, _ = extract_contexts(train_data)
    print(f"Unique training contexts: {len(train_contexts)}")

    print("加载句向量模型...")
    embedder = SentenceTransformer(sentence_transformer_name, device=device)

    print("建立FAISS IVF-PQ索引...")
    index = build_faiss_index_ivfpq(train_contexts, embedder, batch_size=batch_size_embed, nlist=100, m=16, nbits=8)

    print(f"加载生成模型和tokenizer: {model_name}")
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSeq2SeqLM.from_pretrained(model_name).to(device)
    model.eval()

    preds = {}
    max_test = 20
    count = 0

    print("开始对验证集生成回答...")
    for article in dev_data:
        for p in article['paragraphs']:
            for qa in p['qas']:
                if count >= max_test:
                    break
                qid = qa['id']
                question = qa['question']
                retrieved = retrieve_top_k(question, index, embedder, train_contexts, k=5)
                answer = generate_answer(question, retrieved, tokenizer, model)

                preds[qid] = answer

                print(count)
                count += 1
            if count >= max_test:
                break
        if count >= max_test:
            break

    # 构造只包含测试用的子集 dev_data
    filtered_dev_data = []
    for article in dev_data:
        new_article = {"paragraphs": []}
        for p in article['paragraphs']:
            new_p = {"qas": [], "context": p["context"]}
            for qa in p['qas']:
                if qa['id'] in preds:
                    new_p["qas"].append(qa)
            if new_p["qas"]:
                new_article["paragraphs"].append(new_p)
        if new_article["paragraphs"]:
            filtered_dev_data.append(new_article)

    print("计算评测指标...")
    exact_scores, f1_scores = get_raw_scores(filtered_dev_data, preds)
    eval_results = make_eval_dict(exact_scores, f1_scores)

    print("评测结果：")
    for k, v in eval_results.items():
        print(f"{k}: {v:.2f}")

if __name__ == "__main__":
    main()


加载数据集...
提取并去重训练集上下文...
Unique training contexts: 19029
加载句向量模型...


.gitattributes: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

special_tokens_map.json: 0.00B [00:00, ?B/s]

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

开始对验证集生成回答...
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
计算评测指标...
评测结果：
exact: 5.00
f1: 7.50
total: 20.00


## faiss增强-更大QA set

In [5]:
import json
import collections
import re
import string
import faiss
import numpy as np
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from sentence_transformers import SentenceTransformer
from tqdm import tqdm

# ------------------ SQuAD Evaluation Functions ------------------

def normalize_answer(s):
    if s is None:
        return ""
    def remove_articles(text):
        return re.sub(r'\b(a|an|the)\b', ' ', text)
    def white_space_fix(text):
        return ' '.join(text.split())
    def remove_punc(text):
        exclude = set(string.punctuation)
        return ''.join(ch for ch in text if ch not in exclude)
    def lower(text):
        return text.lower()
    return white_space_fix(remove_articles(remove_punc(lower(s))))

def get_tokens(s):
    if not s:
        return []
    return normalize_answer(s).split()

def compute_exact(a_gold, a_pred):
    return int(normalize_answer(a_gold) == normalize_answer(a_pred))

def compute_f1(a_gold, a_pred):
    gold_toks = get_tokens(a_gold)
    pred_toks = get_tokens(a_pred)
    common = collections.Counter(gold_toks) & collections.Counter(pred_toks)
    num_same = sum(common.values())
    if len(gold_toks) == 0 or len(pred_toks) == 0:
        return int(gold_toks == pred_toks)
    if num_same == 0:
        return 0
    precision = 1.0 * num_same / len(pred_toks)
    recall = 1.0 * num_same / len(gold_toks)
    return 2 * precision * recall / (precision + recall)

def get_raw_scores(dataset, preds):
    exact_scores = {}
    f1_scores = {}
    for article in dataset:
        for p in article['paragraphs']:
            for qa in p['qas']:
                qid = qa['id']
                gold_answers = [a['text'] for a in qa.get('answers', []) if normalize_answer(a.get('text', ''))]
                if not gold_answers:
                    gold_answers = ['']  # no answer questions
                a_pred = preds.get(qid, "")
                exact_scores[qid] = max(compute_exact(a, a_pred) for a in gold_answers)
                f1_scores[qid] = max(compute_f1(a, a_pred) for a in gold_answers)
    return exact_scores, f1_scores

def make_eval_dict(exact_scores, f1_scores):
    total = len(exact_scores)
    return collections.OrderedDict([
        ('exact', 100.0 * sum(exact_scores.values()) / total if total else 0),
        ('f1', 100.0 * sum(f1_scores.values()) / total if total else 0),
        ('total', total),
    ])

# ------------------ RAG 主体流程 ------------------

def load_squad_dataset(path):
    with open(path, 'r', encoding='utf-8') as f:
        data_json = json.load(f)
    return data_json['data']

def extract_contexts(dataset):
    contexts = []
    ctx_to_idx = {}
    for article in dataset:
        for p in article['paragraphs']:
            ctx = p['context']
            if ctx not in ctx_to_idx:
                ctx_to_idx[ctx] = len(contexts)
                contexts.append(ctx)
    return contexts, ctx_to_idx

def build_faiss_index_ivfpq(texts, embedder, batch_size=256, nlist=100, m=16, nbits=8):
    # 编码所有文本
    embs = []
    for i in tqdm(range(0, len(texts), batch_size), desc="Embedding batches"):
        batch = texts[i:i+batch_size]
        batch_emb = embedder.encode(batch, convert_to_numpy=True, device='cuda', normalize_embeddings=True, show_progress_bar=False)
        batch_emb = np.asarray(batch_emb, dtype=np.float32)
        embs.append(batch_emb)
    embeddings = np.vstack(embs)

    dim = embeddings.shape[1]
    # IVF + PQ索引
    quantizer = faiss.IndexFlatIP(dim)  # 用内积量化器
    index = faiss.IndexIVFPQ(quantizer, dim, nlist, m, nbits)

    # 训练索引，必须先训练
    print("训练IVF-PQ索引...")
    index.train(embeddings)

    # 添加向量
    print("添加向量到索引...")
    index.add(embeddings)

    return index

def retrieve_top_k(question, index, embedder, contexts, k=5):
    q_emb = embedder.encode([question], convert_to_numpy=True, device='cuda', normalize_embeddings=True, show_progress_bar=False)
    q_emb = np.asarray(q_emb, dtype=np.float32)
    D, I = index.search(q_emb, k)
    hits = []
    for idx in I[0]:
        if 0 <= idx < len(contexts):
            hits.append(contexts[idx])
    return hits

def chunk_and_truncate_contexts(contexts, max_chars=3000):
    out = []
    for c in contexts:
        if len(c) <= max_chars:
            out.append(c)
        else:
            out.append(c[:max_chars] + " ...")
    return out

def generate_answer(question, contexts, tokenizer, model, max_new_tokens=64):
    context_text = "\n".join(chunk_and_truncate_contexts(contexts))
    prompt = f"Question: {question}\nContext:\n{context_text}\nAnswer (if no answer, respond with 'no answer'):"

    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=512)
    device = next(model.parameters()).device
    inputs = {k: v.to(device) for k, v in inputs.items()}

    outputs = model.generate(
        **inputs,
        max_new_tokens=max_new_tokens,
        do_sample=False,
        num_beams=4,
        no_repeat_ngram_size=3,
        early_stopping=True
    )
    answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
    if answer.startswith(prompt):
        answer = answer[len(prompt):]
    answer = answer.strip()
    if answer.lower() in ["false", "no answer", "none", ""]:
        answer = ""
    return answer



def main():
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    train_path = '/kaggle/input/squad-train/train-v2.0.json'
    dev_path = '/kaggle/input/squad-dev/dev-v2.0.json'

    sentence_transformer_name = 'all-mpnet-base-v2'
    model_name = "google/flan-t5-base"
    batch_size_embed = 256

    print("加载数据集...")
    train_data = load_squad_dataset(train_path)
    dev_data = load_squad_dataset(dev_path)

    print("提取并去重训练集上下文...")
    train_contexts, _ = extract_contexts(train_data)
    print(f"Unique training contexts: {len(train_contexts)}")

    print("加载句向量模型...")
    embedder = SentenceTransformer(sentence_transformer_name, device=device)

    print("建立FAISS IVF-PQ索引...")
    index = build_faiss_index_ivfpq(train_contexts, embedder, batch_size=batch_size_embed, nlist=100, m=16, nbits=8)

    print(f"加载生成模型和tokenizer: {model_name}")
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSeq2SeqLM.from_pretrained(model_name).to(device)
    model.eval()

    preds = {}
    max_test = 500  # 改成跑 500 个 QA
    count = 0

    print("开始对验证集生成回答...")
    for article in dev_data:
        for p in article['paragraphs']:
            for qa in p['qas']:
                if count >= max_test:
                    break
                qid = qa['id']
                question = qa['question']
                retrieved = retrieve_top_k(question, index, embedder, train_contexts, k=5)
                answer = generate_answer(question, retrieved, tokenizer, model)
                preds[qid] = answer
                count += 1
                print(count)
            if count >= max_test:
                break
        if count >= max_test:
            break

    # 构造只包含测试用的子集 dev_data
    filtered_dev_data = []
    for article in dev_data:
        new_article = {"paragraphs": []}
        for p in article['paragraphs']:
            new_p = {"qas": [], "context": p["context"]}
            for qa in p['qas']:
                if qa['id'] in preds:  # 只保留前 500 个的 QA
                    new_p["qas"].append(qa)
            if new_p["qas"]:
                new_article["paragraphs"].append(new_p)
        if new_article["paragraphs"]:
            filtered_dev_data.append(new_article)

    print("计算评测指标...")
    exact_scores, f1_scores = get_raw_scores(filtered_dev_data, preds)
    eval_results = make_eval_dict(exact_scores, f1_scores)

    print("评测结果：")
    for k, v in eval_results.items():
        print(f"{k}: {v:.2f}")


if __name__ == "__main__":
    main()


加载数据集...
提取并去重训练集上下文...
Unique training contexts: 19029
加载句向量模型...
建立FAISS IVF-PQ索引...


Embedding batches: 100%|██████████| 75/75 [04:38<00:00,  3.71s/it]


训练IVF-PQ索引...
添加向量到索引...
加载生成模型和tokenizer: google/flan-t5-base
开始对验证集生成回答...
计算评测指标...
评测结果：
exact: 2.20
f1: 3.25
total: 500.00


# 弃用faiss-并提升GPU利用效率

In [8]:
import json
import collections
import re
import string
import numpy as np
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from sentence_transformers import SentenceTransformer
from tqdm import tqdm

# ------------------ SQuAD Evaluation Functions ------------------

def normalize_answer(s):
    if s is None:
        return ""
    def remove_articles(text):
        return re.sub(r'\b(a|an|the)\b', ' ', text)
    def white_space_fix(text):
        return ' '.join(text.split())
    def remove_punc(text):
        exclude = set(string.punctuation)
        return ''.join(ch for ch in text if ch not in exclude)
    def lower(text):
        return text.lower()
    return white_space_fix(remove_articles(remove_punc(lower(s))))

def get_tokens(s):
    if not s:
        return []
    return normalize_answer(s).split()

def compute_exact(a_gold, a_pred):
    return int(normalize_answer(a_gold) == normalize_answer(a_pred))

def compute_f1(a_gold, a_pred):
    gold_toks = get_tokens(a_gold)
    pred_toks = get_tokens(a_pred)
    common = collections.Counter(gold_toks) & collections.Counter(pred_toks)
    num_same = sum(common.values())
    if len(gold_toks) == 0 or len(pred_toks) == 0:
        return int(gold_toks == pred_toks)
    if num_same == 0:
        return 0
    precision = 1.0 * num_same / len(pred_toks)
    recall = 1.0 * num_same / len(gold_toks)
    return 2 * precision * recall / (precision + recall)

def get_raw_scores(dataset, preds):
    exact_scores = {}
    f1_scores = {}
    for article in dataset:
        for p in article['paragraphs']:
            for qa in p['qas']:
                qid = qa['id']
                gold_answers = [a['text'] for a in qa.get('answers', []) if normalize_answer(a.get('text', ''))]
                if not gold_answers:
                    gold_answers = ['']  # no answer questions
                a_pred = preds.get(qid, "")
                exact_scores[qid] = max(compute_exact(a, a_pred) for a in gold_answers)
                f1_scores[qid] = max(compute_f1(a, a_pred) for a in gold_answers)
    return exact_scores, f1_scores

def make_eval_dict(exact_scores, f1_scores):
    total = len(exact_scores)
    return collections.OrderedDict([
        ('exact', 100.0 * sum(exact_scores.values()) / total if total else 0),
        ('f1', 100.0 * sum(f1_scores.values()) / total if total else 0),
        ('total', total),
    ])

# ------------------ RAG 主体流程 ------------------

def load_squad_dataset(path):
    with open(path, 'r', encoding='utf-8') as f:
        data_json = json.load(f)
    return data_json['data']

def chunk_and_truncate_contexts(contexts, max_chars=3000):
    out = []
    for c in contexts:
        if len(c) <= max_chars:
            out.append(c)
        else:
            out.append(c[:max_chars] + " ...")
    return out

def build_context_embeddings(contexts, embedder, batch_size=1024):
    embs = []
    for i in tqdm(range(0, len(contexts), batch_size), desc="Embedding contexts"):
        batch = contexts[i:i+batch_size]
        batch_emb = embedder.encode(batch, convert_to_numpy=True, device='cuda', normalize_embeddings=True, show_progress_bar=False)
        embs.append(batch_emb)
    return np.vstack(embs).astype(np.float32)

def retrieve_top_k(question, all_ctx_emb, embedder, contexts, k=5):
    q_emb = embedder.encode([question], convert_to_numpy=True, device='cuda', normalize_embeddings=True, show_progress_bar=False)
    q_emb = np.asarray(q_emb, dtype=np.float32)
    # 计算所有上下文与问题的内积相似度
    sims = np.dot(all_ctx_emb, q_emb.T).squeeze(1)  # shape: (num_contexts,)
    topk_idx = np.argpartition(-sims, k)[:k]
    # 按相似度排序
    topk_idx = topk_idx[np.argsort(-sims[topk_idx])]
    hits = [contexts[i] for i in topk_idx if i < len(contexts)]
    return hits

def generate_answer(question, contexts, tokenizer, model, max_new_tokens=64):
    context_text = "\n".join(chunk_and_truncate_contexts(contexts))
    prompt = f"Question: {question}\nContext:\n{context_text}\nAnswer (if no answer, respond with 'no answer'):"

    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=512)
    device = next(model.parameters()).device
    inputs = {k: v.to(device) for k, v in inputs.items()}

    outputs = model.generate(
        **inputs,
        max_new_tokens=max_new_tokens,
        do_sample=False,
        num_beams=4,
        no_repeat_ngram_size=3,
        early_stopping=True
    )
    answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
    if answer.startswith(prompt):
        answer = answer[len(prompt):]
    answer = answer.strip()
    if answer.lower() in ["false", "no answer", "none", ""]:
        answer = ""
    return answer

def main():
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    train_file = '/kaggle/input/squad-train/train-v2.0.json'
    dev_file = '/kaggle/input/squad-dev/dev-v2.0.json'

    sentence_transformer_name = 'all-mpnet-base-v2'
    model_name = "google/flan-t5-base"
    batch_size_embed = 256

    print("加载训练数据...")
    train_data = load_squad_dataset(train_file)
    print("加载验证数据...")
    dev_data = load_squad_dataset(dev_file)

    # 提取所有训练上下文（遍历每篇文章每个段落）
    contexts = []
    for article in train_data:
        for para in article["paragraphs"]:
            contexts.append(para["context"])
    print(f"训练集上下文数量: {len(contexts)}")

    print("加载句向量模型...")
    embedder = SentenceTransformer(sentence_transformer_name, device=device)

    print("GPU上计算所有上下文的向量表示...")
    all_ctx_emb = build_context_embeddings(contexts, embedder, batch_size=batch_size_embed)

    print(f"加载生成模型和tokenizer: {model_name}")
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSeq2SeqLM.from_pretrained(model_name).to(device)
    model.eval()

    preds = {}
    max_test = 500
    count = 0

    print(f"开始对验证集前{max_test}个样本生成回答...")
    for article in dev_data:
        for para in article["paragraphs"]:
            for qa in para["qas"]:
                if count >= max_test:
                    break
                qid = qa['id']
                question = qa['question']

                retrieved = retrieve_top_k(question, all_ctx_emb, embedder, contexts, k=5)
                answer = generate_answer(question, retrieved, tokenizer, model)

                preds[qid] = answer

                print(f"{count}: QID={qid}, Answer='{answer[:30]}...'")
                count += 1
            if count >= max_test:
                break
        if count >= max_test:
            break

    # 构造只包含测试用的dev_data子集
    filtered_dev_data = []
    for article in dev_data:
        new_article = {"paragraphs": []}
        for para in article["paragraphs"]:
            new_para = {"qas": [], "context": para["context"]}
            for qa in para["qas"]:
                if qa['id'] in preds:
                    new_para["qas"].append(qa)
            if new_para["qas"]:
                new_article["paragraphs"].append(new_para)
        if new_article["paragraphs"]:
            filtered_dev_data.append(new_article)

    print("计算评测指标...")
    exact_scores, f1_scores = get_raw_scores(filtered_dev_data, preds)
    eval_results = make_eval_dict(exact_scores, f1_scores)

    print("评测结果：")
    for k, v in eval_results.items():
        print(f"{k}: {v:.2f}")

if __name__ == "__main__":
    main()


加载训练数据...
加载验证数据...
训练集上下文数量: 19035
加载句向量模型...
GPU上计算所有上下文的向量表示...


Embedding contexts: 100%|██████████| 75/75 [04:38<00:00,  3.72s/it]


加载生成模型和tokenizer: google/flan-t5-base
开始对验证集前500个样本生成回答...
0: QID=56ddde6b9a695914005b9628, Answer='France...'
1: QID=56ddde6b9a695914005b9629, Answer='11th and 12th centuries...'
2: QID=56ddde6b9a695914005b962a, Answer='Scandinavian...'
3: QID=56ddde6b9a695914005b962b, Answer='King Christian II...'
4: QID=56ddde6b9a695914005b962c, Answer='1066...'
5: QID=5ad39d53604f3c001a3fe8d1, Answer='the Dukes of Normandy...'
6: QID=5ad39d53604f3c001a3fe8d2, Answer='Europe...'
7: QID=5ad39d53604f3c001a3fe8d3, Answer='Lords Proprietors...'
8: QID=5ad39d53604f3c001a3fe8d4, Answer='9th century...'
9: QID=56dddf4066d3e219004dad5f, Answer='Arthur Wellesley took his titl...'
10: QID=56dddf4066d3e219004dad60, Answer='Robert Guiscard...'
11: QID=56dddf4066d3e219004dad61, Answer='Christian...'
12: QID=5ad3a266604f3c001a3fea27, Answer='royal prestige rose to new hei...'
13: QID=5ad3a266604f3c001a3fea28, Answer='Graeco-Roman...'
14: QID=5ad3a266604f3c001a3fea29, Answer='Romance languages...'
15: QID=5ad3a266

## 检索阶段优化，增大k与更换效果更好的句向量模型

In [12]:
import json
import collections
import re
import string
import numpy as np
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from sentence_transformers import SentenceTransformer
from tqdm import tqdm

# ------------------ SQuAD Evaluation Functions ------------------

def normalize_answer(s):
    if s is None:
        return ""
    def remove_articles(text):
        return re.sub(r'\b(a|an|the)\b', ' ', text)
    def white_space_fix(text):
        return ' '.join(text.split())
    def remove_punc(text):
        exclude = set(string.punctuation)
        return ''.join(ch for ch in text if ch not in exclude)
    def lower(text):
        return text.lower()
    return white_space_fix(remove_articles(remove_punc(lower(s))))

def get_tokens(s):
    if not s:
        return []
    return normalize_answer(s).split()

def compute_exact(a_gold, a_pred):
    return int(normalize_answer(a_gold) == normalize_answer(a_pred))

def compute_f1(a_gold, a_pred):
    gold_toks = get_tokens(a_gold)
    pred_toks = get_tokens(a_pred)
    common = collections.Counter(gold_toks) & collections.Counter(pred_toks)
    num_same = sum(common.values())
    if len(gold_toks) == 0 or len(pred_toks) == 0:
        return int(gold_toks == pred_toks)
    if num_same == 0:
        return 0
    precision = 1.0 * num_same / len(pred_toks)
    recall = 1.0 * num_same / len(gold_toks)
    return 2 * precision * recall / (precision + recall)

def get_raw_scores(dataset, preds):
    exact_scores = {}
    f1_scores = {}
    for article in dataset:
        for p in article['paragraphs']:
            for qa in p['qas']:
                qid = qa['id']
                gold_answers = [a['text'] for a in qa.get('answers', []) if normalize_answer(a.get('text', ''))]
                if not gold_answers:
                    gold_answers = ['']  # no answer questions
                a_pred = preds.get(qid, "")
                exact_scores[qid] = max(compute_exact(a, a_pred) for a in gold_answers)
                f1_scores[qid] = max(compute_f1(a, a_pred) for a in gold_answers)
    return exact_scores, f1_scores

def make_eval_dict(exact_scores, f1_scores):
    total = len(exact_scores)
    return collections.OrderedDict([
        ('exact', 100.0 * sum(exact_scores.values()) / total if total else 0),
        ('f1', 100.0 * sum(f1_scores.values()) / total if total else 0),
        ('total', total),
    ])

# ------------------ RAG 主体流程 ------------------

def load_squad_dataset(path):
    with open(path, 'r', encoding='utf-8') as f:
        data_json = json.load(f)
    return data_json['data']

def chunk_and_truncate_contexts(contexts, max_chars=3000):
    out = []
    for c in contexts:
        if len(c) <= max_chars:
            out.append(c)
        else:
            out.append(c[:max_chars] + " ...")
    return out

def build_context_embeddings(contexts, embedder, batch_size=512):
    embs = []
    for i in tqdm(range(0, len(contexts), batch_size), desc="Embedding contexts"):
        batch = contexts[i:i+batch_size]
        batch_emb = embedder.encode(batch, convert_to_numpy=True, device='cuda', normalize_embeddings=True, show_progress_bar=False)
        batch_emb = np.asarray(batch_emb, dtype=np.float32)
        embs.append(batch_emb)
    embedder.to('cpu')  # 回收GPU显存
    return np.vstack(embs)


def retrieve_top_k(question, all_ctx_emb, embedder, contexts, k=20):
    q_emb = embedder.encode([question], convert_to_numpy=True, device='cuda', normalize_embeddings=True, show_progress_bar=False)
    q_emb = np.asarray(q_emb, dtype=np.float32)
    # 计算所有上下文与问题的内积相似度
    sims = np.dot(all_ctx_emb, q_emb.T).squeeze(1)  # shape: (num_contexts,)
    topk_idx = np.argpartition(-sims, k)[:k]
    # 按相似度排序
    topk_idx = topk_idx[np.argsort(-sims[topk_idx])]
    hits = [contexts[i] for i in topk_idx if i < len(contexts)]
    return hits

def generate_answer(question, contexts, tokenizer, model, max_new_tokens=64):
    context_text = "\n".join(chunk_and_truncate_contexts(contexts))
    prompt = f"Question: {question}\nContext:\n{context_text}\nAnswer (if no answer, respond with 'no answer'):"

    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=512)
    device = next(model.parameters()).device
    inputs = {k: v.to(device) for k, v in inputs.items()}

    outputs = model.generate(
        **inputs,
        max_new_tokens=max_new_tokens,
        do_sample=False,
        num_beams=4,
        no_repeat_ngram_size=3,
        early_stopping=True
    )
    answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
    if answer.startswith(prompt):
        answer = answer[len(prompt):]
    answer = answer.strip()
    if answer.lower() in ["false", "no answer", "none", ""]:
        answer = ""
    return answer

def main():
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    train_file = '/kaggle/input/squad-train/train-v2.0.json'
    dev_file = '/kaggle/input/squad-dev/dev-v2.0.json'

    sentence_transformer_name = 'sentence-transformers/all-mpnet-base-v2'
    model_name = "google/flan-t5-base"
    batch_size_embed = 256

    print("加载训练数据...")
    train_data = load_squad_dataset(train_file)
    print("加载验证数据...")
    dev_data = load_squad_dataset(dev_file)

    # 提取所有训练上下文（遍历每篇文章每个段落）
    contexts = []
    for article in train_data:
        for para in article["paragraphs"]:
            contexts.append(para["context"])
    print(f"训练集上下文数量: {len(contexts)}")

    print("加载句向量模型...")
    embedder = SentenceTransformer(sentence_transformer_name, device=device)

    print("GPU上计算所有上下文的向量表示...")
    all_ctx_emb = build_context_embeddings(contexts, embedder, batch_size=batch_size_embed)

    print(f"加载生成模型和tokenizer: {model_name}")
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSeq2SeqLM.from_pretrained(model_name).to(device)
    model.eval()

    preds = {}
    max_test = 500
    count = 0

    print(f"开始对验证集前{max_test}个样本生成回答...")
    for article in dev_data:
        for para in article["paragraphs"]:
            for qa in para["qas"]:
                if count >= max_test:
                    break
                qid = qa['id']
                question = qa['question']

                retrieved = retrieve_top_k(question, all_ctx_emb, embedder, contexts, k=5)
                answer = generate_answer(question, retrieved, tokenizer, model)

                preds[qid] = answer

                print(count)
                count += 1
            if count >= max_test:
                break
        if count >= max_test:
            break

    # 构造只包含测试用的dev_data子集
    filtered_dev_data = []
    for article in dev_data:
        new_article = {"paragraphs": []}
        for para in article["paragraphs"]:
            new_para = {"qas": [], "context": para["context"]}
            for qa in para["qas"]:
                if qa['id'] in preds:
                    new_para["qas"].append(qa)
            if new_para["qas"]:
                new_article["paragraphs"].append(new_para)
        if new_article["paragraphs"]:
            filtered_dev_data.append(new_article)

    print("计算评测指标...")
    exact_scores, f1_scores = get_raw_scores(filtered_dev_data, preds)
    eval_results = make_eval_dict(exact_scores, f1_scores)

    print("评测结果：")
    for k, v in eval_results.items():
        print(f"{k}: {v:.2f}")

if __name__ == "__main__":
    main()


加载训练数据...
加载验证数据...
训练集上下文数量: 19035
加载句向量模型...
GPU上计算所有上下文的向量表示...


Embedding contexts: 100%|██████████| 75/75 [04:39<00:00,  3.72s/it]


加载生成模型和tokenizer: google/flan-t5-base
开始对验证集前500个样本生成回答...
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262

# 更换google生成模型

In [13]:
import json
import collections
import re
import string
import numpy as np
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from sentence_transformers import SentenceTransformer
from tqdm import tqdm

# ------------------ SQuAD Evaluation Functions ------------------

def normalize_answer(s):
    if s is None:
        return ""
    def remove_articles(text):
        return re.sub(r'\b(a|an|the)\b', ' ', text)
    def white_space_fix(text):
        return ' '.join(text.split())
    def remove_punc(text):
        exclude = set(string.punctuation)
        return ''.join(ch for ch in text if ch not in exclude)
    def lower(text):
        return text.lower()
    return white_space_fix(remove_articles(remove_punc(lower(s))))

def get_tokens(s):
    if not s:
        return []
    return normalize_answer(s).split()

def compute_exact(a_gold, a_pred):
    return int(normalize_answer(a_gold) == normalize_answer(a_pred))

def compute_f1(a_gold, a_pred):
    gold_toks = get_tokens(a_gold)
    pred_toks = get_tokens(a_pred)
    common = collections.Counter(gold_toks) & collections.Counter(pred_toks)
    num_same = sum(common.values())
    if len(gold_toks) == 0 or len(pred_toks) == 0:
        return int(gold_toks == pred_toks)
    if num_same == 0:
        return 0
    precision = 1.0 * num_same / len(pred_toks)
    recall = 1.0 * num_same / len(gold_toks)
    return 2 * precision * recall / (precision + recall)

def get_raw_scores(dataset, preds):
    exact_scores = {}
    f1_scores = {}
    for article in dataset:
        for p in article['paragraphs']:
            for qa in p['qas']:
                qid = qa['id']
                gold_answers = [a['text'] for a in qa.get('answers', []) if normalize_answer(a.get('text', ''))]
                if not gold_answers:
                    gold_answers = ['']  # no answer questions
                a_pred = preds.get(qid, "")
                exact_scores[qid] = max(compute_exact(a, a_pred) for a in gold_answers)
                f1_scores[qid] = max(compute_f1(a, a_pred) for a in gold_answers)
    return exact_scores, f1_scores

def make_eval_dict(exact_scores, f1_scores):
    total = len(exact_scores)
    return collections.OrderedDict([
        ('exact', 100.0 * sum(exact_scores.values()) / total if total else 0),
        ('f1', 100.0 * sum(f1_scores.values()) / total if total else 0),
        ('total', total),
    ])

# ------------------ RAG 主体流程 ------------------

def load_squad_dataset(path):
    with open(path, 'r', encoding='utf-8') as f:
        data_json = json.load(f)
    return data_json['data']

def chunk_and_truncate_contexts(contexts, max_chars=3000):
    out = []
    for c in contexts:
        if len(c) <= max_chars:
            out.append(c)
        else:
            out.append(c[:max_chars] + " ...")
    return out

def build_context_embeddings(contexts, embedder, batch_size=512):
    embs = []
    for i in tqdm(range(0, len(contexts), batch_size), desc="Embedding contexts"):
        batch = contexts[i:i+batch_size]
        batch_emb = embedder.encode(batch, convert_to_numpy=True, device='cuda', normalize_embeddings=True, show_progress_bar=False)
        batch_emb = np.asarray(batch_emb, dtype=np.float32)
        embs.append(batch_emb)
    embedder.to('cpu')  # 回收GPU显存
    return np.vstack(embs)


def retrieve_top_k(question, all_ctx_emb, embedder, contexts, k=20):
    q_emb = embedder.encode([question], convert_to_numpy=True, device='cuda', normalize_embeddings=True, show_progress_bar=False)
    q_emb = np.asarray(q_emb, dtype=np.float32)
    # 计算所有上下文与问题的内积相似度
    sims = np.dot(all_ctx_emb, q_emb.T).squeeze(1)  # shape: (num_contexts,)
    topk_idx = np.argpartition(-sims, k)[:k]
    # 按相似度排序
    topk_idx = topk_idx[np.argsort(-sims[topk_idx])]
    hits = [contexts[i] for i in topk_idx if i < len(contexts)]
    return hits

def generate_answer(question, contexts, tokenizer, model, max_new_tokens=64):
    context_text = "\n".join(chunk_and_truncate_contexts(contexts))
    prompt = f"Question: {question}\nContext:\n{context_text}\nAnswer (if no answer, respond with 'no answer'):"

    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=512)
    device = next(model.parameters()).device
    inputs = {k: v.to(device) for k, v in inputs.items()}

    outputs = model.generate(
        **inputs,
        max_new_tokens=max_new_tokens,
        do_sample=False,
        num_beams=4,
        no_repeat_ngram_size=3,
        early_stopping=True
    )
    answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
    if answer.startswith(prompt):
        answer = answer[len(prompt):]
    answer = answer.strip()
    if answer.lower() in ["false", "no answer", "none", ""]:
        answer = ""
    return answer

def main():
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    train_file = '/kaggle/input/squad-train/train-v2.0.json'
    dev_file = '/kaggle/input/squad-dev/dev-v2.0.json'

    sentence_transformer_name = 'sentence-transformers/all-mpnet-base-v2'
    model_name = "google/flan-t5-xl"
    batch_size_embed = 256

    print("加载训练数据...")
    train_data = load_squad_dataset(train_file)
    print("加载验证数据...")
    dev_data = load_squad_dataset(dev_file)

    # 提取所有训练上下文（遍历每篇文章每个段落）
    contexts = []
    for article in train_data:
        for para in article["paragraphs"]:
            contexts.append(para["context"])
    print(f"训练集上下文数量: {len(contexts)}")

    print("加载句向量模型...")
    embedder = SentenceTransformer(sentence_transformer_name, device=device)

    print("GPU上计算所有上下文的向量表示...")
    all_ctx_emb = build_context_embeddings(contexts, embedder, batch_size=batch_size_embed)

    print(f"加载生成模型和tokenizer: {model_name}")
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSeq2SeqLM.from_pretrained(model_name).to(device)
    model.eval()

    preds = {}
    max_test = 500
    count = 0

    print(f"开始对验证集前{max_test}个样本生成回答...")
    for article in dev_data:
        for para in article["paragraphs"]:
            for qa in para["qas"]:
                if count >= max_test:
                    break
                qid = qa['id']
                question = qa['question']

                retrieved = retrieve_top_k(question, all_ctx_emb, embedder, contexts, k=5)
                answer = generate_answer(question, retrieved, tokenizer, model)

                preds[qid] = answer

                print(count)
                count += 1
            if count >= max_test:
                break
        if count >= max_test:
            break

    # 构造只包含测试用的dev_data子集
    filtered_dev_data = []
    for article in dev_data:
        new_article = {"paragraphs": []}
        for para in article["paragraphs"]:
            new_para = {"qas": [], "context": para["context"]}
            for qa in para["qas"]:
                if qa['id'] in preds:
                    new_para["qas"].append(qa)
            if new_para["qas"]:
                new_article["paragraphs"].append(new_para)
        if new_article["paragraphs"]:
            filtered_dev_data.append(new_article)

    print("计算评测指标...")
    exact_scores, f1_scores = get_raw_scores(filtered_dev_data, preds)
    eval_results = make_eval_dict(exact_scores, f1_scores)

    print("评测结果：")
    for k, v in eval_results.items():
        print(f"{k}: {v:.2f}")

if __name__ == "__main__":
    main()


加载训练数据...
加载验证数据...
训练集上下文数量: 19035
加载句向量模型...
GPU上计算所有上下文的向量表示...


Embedding contexts: 100%|██████████| 75/75 [04:39<00:00,  3.72s/it]


加载生成模型和tokenizer: google/flan-t5-xl


tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/9.45G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.95G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

开始对验证集前500个样本生成回答...
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
2

## 使用hugging face多卡并行并尝试调节生成模型端参数
多卡失败，尝试单卡调参

In [ ]:
import json
import collections
import re
import string
import numpy as np
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from sentence_transformers import SentenceTransformer
from tqdm import tqdm
from accelerate import Accelerator

# ------------------ SQuAD 评测函数 ------------------

def normalize_answer(s):
    if s is None:
        return ""
    def remove_articles(text):
        return re.sub(r'\b(a|an|the)\b', ' ', text)
    def white_space_fix(text):
        return ' '.join(text.split())
    def remove_punc(text):
        exclude = set(string.punctuation)
        return ''.join(ch for ch in text if ch not in exclude)
    def lower(text):
        return text.lower()
    return white_space_fix(remove_articles(remove_punc(lower(s))))

def get_tokens(s):
    if not s:
        return []
    return normalize_answer(s).split()

def compute_exact(a_gold, a_pred):
    return int(normalize_answer(a_gold) == normalize_answer(a_pred))

def compute_f1(a_gold, a_pred):
    gold_toks = get_tokens(a_gold)
    pred_toks = get_tokens(a_pred)
    common = collections.Counter(gold_toks) & collections.Counter(pred_toks)
    num_same = sum(common.values())
    if len(gold_toks) == 0 or len(pred_toks) == 0:
        return int(gold_toks == pred_toks)
    if num_same == 0:
        return 0
    precision = 1.0 * num_same / len(pred_toks)
    recall = 1.0 * num_same / len(gold_toks)
    return 2 * precision * recall / (precision + recall)

def get_raw_scores(dataset, preds):
    exact_scores = {}
    f1_scores = {}
    for article in dataset:
        for p in article['paragraphs']:
            for qa in p['qas']:
                qid = qa['id']
                gold_answers = [a['text'] for a in qa.get('answers', []) if normalize_answer(a.get('text', ''))]
                if not gold_answers:
                    gold_answers = ['']  # no answer questions
                a_pred = preds.get(qid, "")
                exact_scores[qid] = max(compute_exact(a, a_pred) for a in gold_answers)
                f1_scores[qid] = max(compute_f1(a, a_pred) for a in gold_answers)
    return exact_scores, f1_scores

def make_eval_dict(exact_scores, f1_scores):
    total = len(exact_scores)
    return collections.OrderedDict([
        ('exact', 100.0 * sum(exact_scores.values()) / total if total else 0),
        ('f1', 100.0 * sum(f1_scores.values()) / total if total else 0),
        ('total', total),
    ])

# ------------------ 数据加载与上下文处理 ------------------

def load_squad_dataset(path):
    with open(path, 'r', encoding='utf-8') as f:
        data_json = json.load(f)
    return data_json['data']

def chunk_and_truncate_contexts(contexts, max_chars=3000):
    out = []
    for c in contexts:
        if len(c) <= max_chars:
            out.append(c)
        else:
            out.append(c[:max_chars] + " ...")
    return out

def build_context_embeddings(contexts, embedder, batch_size=256):
    embs = []
    for i in tqdm(range(0, len(contexts), batch_size), desc="Embedding contexts"):
        batch = contexts[i:i+batch_size]
        batch_emb = embedder.encode(batch, convert_to_numpy=True, device='cuda', normalize_embeddings=True, show_progress_bar=False)
        batch_emb = np.asarray(batch_emb, dtype=np.float32)
        embs.append(batch_emb)
    return np.vstack(embs)

def retrieve_top_k(question, all_ctx_emb, embedder, contexts, k=20):
    q_emb = embedder.encode([question], convert_to_numpy=True, device='cuda', normalize_embeddings=True, show_progress_bar=False)
    q_emb = np.asarray(q_emb, dtype=np.float32)
    sims = np.dot(all_ctx_emb, q_emb.T).squeeze(1)  # shape: (num_contexts,)
    topk_idx = np.argpartition(-sims, k)[:k]
    topk_idx = topk_idx[np.argsort(-sims[topk_idx])]
    hits = [contexts[i] for i in topk_idx if i < len(contexts)]
    return hits

# ------------------ 生成答案 ------------------

def get_first_device_from_device_map(device_map):
    # 先找所有设备索引（无论单个数字还是列表）
    devices = []
    for v in device_map.values():
        if isinstance(v, list):
            devices.extend(v)
        else:
            devices.append(v)
    # 取最小设备号（通常是第一个GPU）
    first_device_idx = min(devices)
    return torch.device(f'cuda:{first_device_idx}')

def generate_answer(question, contexts, tokenizer, model, accelerator, gen_params):
    context_text = "\n".join(chunk_and_truncate_contexts(contexts))
    prompt = f"Question: {question}\nContext:\n{context_text}\nAnswer (if no answer, respond with 'no answer'):"

    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=512)

    if hasattr(model, 'device_map') and model.device_map:
        device_for_input = get_first_device_from_device_map(model.device_map)
    else:
        device_for_input = accelerator.device

    inputs = {k: v.to(device_for_input) for k, v in inputs.items()}

    with torch.no_grad():
        with accelerator.autocast():
            outputs = model.generate(
                **inputs,
                max_new_tokens=gen_params.get("max_new_tokens", 80),
                do_sample=gen_params.get("do_sample", False),
                num_beams=gen_params.get("num_beams", 6),
                no_repeat_ngram_size=gen_params.get("no_repeat_ngram_size", 3),
                early_stopping=gen_params.get("early_stopping", True),
                temperature=gen_params.get("temperature", 1.0),
                top_k=gen_params.get("top_k", None),
                top_p=gen_params.get("top_p", None),
            )
    answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
    if answer.startswith(prompt):
        answer = answer[len(prompt):]
    answer = answer.strip()
    if answer.lower() in ["false", "no answer", "none", ""]:
        answer = ""
    return answer




# ------------------ 主流程 ------------------

def main():
    accelerator = Accelerator(mixed_precision='fp16')

    train_file = '/kaggle/input/squad-train/train-v2.0.json'
    dev_file = '/kaggle/input/squad-dev/dev-v2.0.json'

    sentence_transformer_name = 'sentence-transformers/all-mpnet-base-v2'
    model_name = "google/flan-t5-xl"
    batch_size_embed = 256

    print("加载训练数据...")
    train_data = load_squad_dataset(train_file)
    print("加载验证数据...")
    dev_data = load_squad_dataset(dev_file)

    contexts = []
    for article in train_data:
        for para in article["paragraphs"]:
            contexts.append(para["context"])
    print(f"训练集上下文数量: {len(contexts)}")

    print("加载句向量模型（GPU）...")
    embedder = SentenceTransformer(sentence_transformer_name, device='cuda')

    print("计算所有上下文向量（GPU）...")
    all_ctx_emb = build_context_embeddings(contexts, embedder, batch_size=batch_size_embed)

    print(f"加载生成模型和tokenizer: {model_name}")
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

    n_gpus = torch.cuda.device_count()
    print(f"检测到 {n_gpus} 张GPU")
    if n_gpus >= 2:
        print("启用模型并行，将模型拆分到多张GPU")
        model.parallelize()
    else:
        print("单卡模式，将模型加载到CUDA设备")
        model = model.to('cuda')

    model, tokenizer = accelerator.prepare(model, tokenizer)
    model.eval()

    preds = {}
    max_test = 500
    count = 0

    gen_params = {
        "max_new_tokens": 80,
        "num_beams": 6,
        "do_sample": False,
        "no_repeat_ngram_size": 3,
        "early_stopping": True,
        "temperature": 1.0,
        "top_k": None,
        "top_p": None,
    }

    print(f"开始对验证集前{max_test}个样本生成回答...")
    for article in dev_data:
        for para in article["paragraphs"]:
            for qa in para["qas"]:
                if count >= max_test:
                    break
                qid = qa['id']
                question = qa['question']

                retrieved = retrieve_top_k(question, all_ctx_emb, embedder, contexts, k=20)
                answer = generate_answer(question, retrieved, tokenizer, model, accelerator, gen_params)

                preds[qid] = answer

                if count % 20 == 0:
                    print(f"已处理 {count} 个问题")
                count += 1
            if count >= max_test:
                break
        if count >= max_test:
            break

    filtered_dev_data = []
    for article in dev_data:
        new_article = {"paragraphs": []}
        for para in article['paragraphs']:
            new_para = {"qas": [], "context": para["context"]}
            for qa in para["qas"]:
                if qa['id'] in preds:
                    new_para["qas"].append(qa)
            if new_para["qas"]:
                new_article["paragraphs"].append(new_para)
        if new_article["paragraphs"]:
            filtered_dev_data.append(new_article)

    print("计算评测指标...")
    exact_scores, f1_scores = get_raw_scores(filtered_dev_data, preds)
    eval_results = make_eval_dict(exact_scores, f1_scores)

    print("评测结果：")
    for k, v in eval_results.items():
        print(f"{k}: {v:.2f}")

if __name__ == "__main__":
    main()


### 单卡调参

In [1]:
import json
import collections
import re
import string
import numpy as np
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from sentence_transformers import SentenceTransformer
from tqdm import tqdm

# ------------------ SQuAD 评测函数 ------------------

def normalize_answer(s):
    if s is None:
        return ""
    def remove_articles(text):
        return re.sub(r'\b(a|an|the)\b', ' ', text)
    def white_space_fix(text):
        return ' '.join(text.split())
    def remove_punc(text):
        exclude = set(string.punctuation)
        return ''.join(ch for ch in text if ch not in exclude)
    def lower(text):
        return text.lower()
    return white_space_fix(remove_articles(remove_punc(lower(s))))

def get_tokens(s):
    if not s:
        return []
    return normalize_answer(s).split()

def compute_exact(a_gold, a_pred):
    return int(normalize_answer(a_gold) == normalize_answer(a_pred))

def compute_f1(a_gold, a_pred):
    gold_toks = get_tokens(a_gold)
    pred_toks = get_tokens(a_pred)
    common = collections.Counter(gold_toks) & collections.Counter(pred_toks)
    num_same = sum(common.values())
    if len(gold_toks) == 0 or len(pred_toks) == 0:
        return int(gold_toks == pred_toks)
    if num_same == 0:
        return 0
    precision = 1.0 * num_same / len(pred_toks)
    recall = 1.0 * num_same / len(gold_toks)
    return 2 * precision * recall / (precision + recall)

def get_raw_scores(dataset, preds):
    exact_scores = {}
    f1_scores = {}
    for article in dataset:
        for p in article['paragraphs']:
            for qa in p['qas']:
                qid = qa['id']
                gold_answers = [a['text'] for a in qa.get('answers', []) if normalize_answer(a.get('text', ''))]
                if not gold_answers:
                    gold_answers = ['']  # no answer questions
                a_pred = preds.get(qid, "")
                exact_scores[qid] = max(compute_exact(a, a_pred) for a in gold_answers)
                f1_scores[qid] = max(compute_f1(a, a_pred) for a in gold_answers)
    return exact_scores, f1_scores

def make_eval_dict(exact_scores, f1_scores):
    total = len(exact_scores)
    return collections.OrderedDict([
        ('exact', 100.0 * sum(exact_scores.values()) / total if total else 0),
        ('f1', 100.0 * sum(f1_scores.values()) / total if total else 0),
        ('total', total),
    ])

# ------------------ 数据加载与上下文处理 ------------------

def load_squad_dataset(path):
    with open(path, 'r', encoding='utf-8') as f:
        data_json = json.load(f)
    return data_json['data']

def chunk_and_truncate_contexts(contexts, max_chars=3000):
    out = []
    for c in contexts:
        if len(c) <= max_chars:
            out.append(c)
        else:
            out.append(c[:max_chars] + " ...")
    return out

def build_context_embeddings(contexts, embedder, batch_size=128):
    embs = []
    for i in tqdm(range(0, len(contexts), batch_size), desc="Embedding contexts"):
        batch = contexts[i:i+batch_size]
        batch_emb = embedder.encode(batch, convert_to_numpy=True, device='cuda', normalize_embeddings=True, show_progress_bar=False)
        batch_emb = np.asarray(batch_emb, dtype=np.float32)
        embs.append(batch_emb)
    return np.vstack(embs)

def retrieve_top_k(question, all_ctx_emb, embedder, contexts, k=10):
    q_emb = embedder.encode([question], convert_to_numpy=True, device='cuda', normalize_embeddings=True, show_progress_bar=False)
    q_emb = np.asarray(q_emb, dtype=np.float32)
    sims = np.dot(all_ctx_emb, q_emb.T).squeeze(1)  # shape: (num_contexts,)
    topk_idx = np.argpartition(-sims, k)[:k]
    topk_idx = topk_idx[np.argsort(-sims[topk_idx])]
    hits = [contexts[i] for i in topk_idx if i < len(contexts)]
    return hits

# ------------------ 生成答案 ------------------

def generate_answer(question, contexts, tokenizer, model, gen_params):
    context_text = "\n".join(chunk_and_truncate_contexts(contexts))
    prompt = f"Question: {question}\nContext:\n{context_text}\nAnswer (if no answer, respond with 'no answer'):"

    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=512).to('cuda')

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=gen_params.get("max_new_tokens", 40),
            do_sample=gen_params.get("do_sample", False),
            num_beams=gen_params.get("num_beams", 2),
            no_repeat_ngram_size=gen_params.get("no_repeat_ngram_size", 3),
            early_stopping=gen_params.get("early_stopping", True),
        )
    answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
    if answer.startswith(prompt):
        answer = answer[len(prompt):]
    answer = answer.strip()
    if answer.lower() in ["false", "no answer", "none", ""]:
        answer = ""
    return answer

# ------------------ 主流程 ------------------

def main():
    train_file = '/kaggle/input/squad-train/train-v2.0.json'
    dev_file = '/kaggle/input/squad-dev/dev-v2.0.json'

    sentence_transformer_name = 'sentence-transformers/all-mpnet-base-v2'
    model_name = "google/flan-t5-xl"

    batch_size_embed = 128
    max_test = 500

    print("加载训练数据...")
    train_data = load_squad_dataset(train_file)
    print("加载验证数据...")
    dev_data = load_squad_dataset(dev_file)

    contexts = []
    for article in train_data:
        for para in article["paragraphs"]:
            contexts.append(para["context"])
    print(f"训练集上下文数量: {len(contexts)}")

    print("加载句向量模型（GPU）...")
    embedder = SentenceTransformer(sentence_transformer_name, device='cuda')

    print("计算所有上下文向量（GPU）...")
    all_ctx_emb = build_context_embeddings(contexts, embedder, batch_size=batch_size_embed)

    print(f"加载生成模型和tokenizer: {model_name}")
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSeq2SeqLM.from_pretrained(model_name).to('cuda')

    model.eval()
    preds = {}

    gen_params = {
        "max_new_tokens": 40,
        "num_beams": 6,
        "do_sample": False,
        "no_repeat_ngram_size": 3,
        "early_stopping": True,
    }

    print(f"开始对验证集前{max_test}个样本生成回答...")
    count = 0
    for article in dev_data:
        for para in article["paragraphs"]:
            for qa in para["qas"]:
                if count >= max_test:
                    break
                qid = qa['id']
                question = qa['question']

                retrieved = retrieve_top_k(question, all_ctx_emb, embedder, contexts, k=10)
                answer = generate_answer(question, retrieved, tokenizer, model, gen_params)

                preds[qid] = answer

                if count % 10 == 0:
                    print(f"已处理 {count} 个问题")
                count += 1
            if count >= max_test:
                break
        if count >= max_test:
            break

    filtered_dev_data = []
    for article in dev_data:
        new_article = {"paragraphs": []}
        for para in article['paragraphs']:
            new_para = {"qas": [], "context": para["context"]}
            for qa in para["qas"]:
                if qa['id'] in preds:
                    new_para["qas"].append(qa)
            if new_para["qas"]:
                new_article["paragraphs"].append(new_para)
        if new_article["paragraphs"]:
            filtered_dev_data.append(new_article)

    print("计算评测指标...")
    exact_scores, f1_scores = get_raw_scores(filtered_dev_data, preds)
    eval_results = make_eval_dict(exact_scores, f1_scores)

    print("评测结果：")
    for k, v in eval_results.items():
        print(f"{k}: {v:.2f}")

if __name__ == "__main__":
    main()


加载训练数据...
加载验证数据...
训练集上下文数量: 19035
加载句向量模型（GPU）...
计算所有上下文向量（GPU）...


Embedding contexts: 100%|██████████| 149/149 [04:59<00:00,  2.01s/it]
/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


加载生成模型和tokenizer: google/flan-t5-xl


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

开始对验证集前500个样本生成回答...
已处理 0 个问题
已处理 10 个问题
已处理 20 个问题
已处理 30 个问题
已处理 40 个问题
已处理 50 个问题
已处理 60 个问题
已处理 70 个问题
已处理 80 个问题
已处理 90 个问题
已处理 100 个问题
已处理 110 个问题
已处理 120 个问题
已处理 130 个问题
已处理 140 个问题
已处理 150 个问题
已处理 160 个问题
已处理 170 个问题
已处理 180 个问题
已处理 190 个问题
已处理 200 个问题
已处理 210 个问题
已处理 220 个问题
已处理 230 个问题
已处理 240 个问题
已处理 250 个问题
已处理 260 个问题
已处理 270 个问题
已处理 280 个问题
已处理 290 个问题
已处理 300 个问题
已处理 310 个问题
已处理 320 个问题
已处理 330 个问题
已处理 340 个问题
已处理 350 个问题
已处理 360 个问题
已处理 370 个问题
已处理 380 个问题
已处理 390 个问题
已处理 400 个问题
已处理 410 个问题
已处理 420 个问题
已处理 430 个问题
已处理 440 个问题
已处理 450 个问题
已处理 460 个问题
已处理 470 个问题
已处理 480 个问题
已处理 490 个问题
计算评测指标...
评测结果：
exact: 1.60
f1: 4.41
total: 500.00


第一次调参效果变差，
考虑max_new_tokens 太短了
40个token可能不足以生成完整答案，尤其复杂问题。缩短生成长度容易导致答案不完整。

上下文检索k增加，带入无关或噪声
检索更多上下文，理论上有更多信息，但也可能带入不相关内容，干扰模型生成。

num_beams 增大未必提升
增大beam宽度能探索更多可能，但如果输入上下文信息有噪声，beam越大可能越倾向于产生受干扰的答案。

模型对上下文的长度敏感
如果输入prompt超长，模型截断导致有用信息丢失，也会影响结果。

## 

## 第二次调参

k = 8,num beams = 5 max token = 80,第二次调参

In [1]:
import json
import collections
import re
import string
import numpy as np
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from sentence_transformers import SentenceTransformer
from tqdm import tqdm

# ------------------ SQuAD 评测函数 ------------------

def normalize_answer(s):
    if s is None:
        return ""
    def remove_articles(text):
        return re.sub(r'\b(a|an|the)\b', ' ', text)
    def white_space_fix(text):
        return ' '.join(text.split())
    def remove_punc(text):
        exclude = set(string.punctuation)
        return ''.join(ch for ch in text if ch not in exclude)
    def lower(text):
        return text.lower()
    return white_space_fix(remove_articles(remove_punc(lower(s))))

def get_tokens(s):
    if not s:
        return []
    return normalize_answer(s).split()

def compute_exact(a_gold, a_pred):
    return int(normalize_answer(a_gold) == normalize_answer(a_pred))

def compute_f1(a_gold, a_pred):
    gold_toks = get_tokens(a_gold)
    pred_toks = get_tokens(a_pred)
    common = collections.Counter(gold_toks) & collections.Counter(pred_toks)
    num_same = sum(common.values())
    if len(gold_toks) == 0 or len(pred_toks) == 0:
        return int(gold_toks == pred_toks)
    if num_same == 0:
        return 0
    precision = 1.0 * num_same / len(pred_toks)
    recall = 1.0 * num_same / len(gold_toks)
    return 2 * precision * recall / (precision + recall)

def get_raw_scores(dataset, preds):
    exact_scores = {}
    f1_scores = {}
    for article in dataset:
        for p in article['paragraphs']:
            for qa in p['qas']:
                qid = qa['id']
                gold_answers = [a['text'] for a in qa.get('answers', []) if normalize_answer(a.get('text', ''))]
                if not gold_answers:
                    gold_answers = ['']  # no answer questions
                a_pred = preds.get(qid, "")
                exact_scores[qid] = max(compute_exact(a, a_pred) for a in gold_answers)
                f1_scores[qid] = max(compute_f1(a, a_pred) for a in gold_answers)
    return exact_scores, f1_scores

def make_eval_dict(exact_scores, f1_scores):
    total = len(exact_scores)
    return collections.OrderedDict([
        ('exact', 100.0 * sum(exact_scores.values()) / total if total else 0),
        ('f1', 100.0 * sum(f1_scores.values()) / total if total else 0),
        ('total', total),
    ])

# ------------------ 数据加载与上下文处理 ------------------

def load_squad_dataset(path):
    with open(path, 'r', encoding='utf-8') as f:
        data_json = json.load(f)
    return data_json['data']

def chunk_and_truncate_contexts(contexts, max_chars=3000):
    out = []
    for c in contexts:
        if len(c) <= max_chars:
            out.append(c)
        else:
            out.append(c[:max_chars] + " ...")
    return out

def build_context_embeddings(contexts, embedder, batch_size=128):
    embs = []
    for i in tqdm(range(0, len(contexts), batch_size), desc="Embedding contexts"):
        batch = contexts[i:i+batch_size]
        batch_emb = embedder.encode(batch, convert_to_numpy=True, device='cuda', normalize_embeddings=True, show_progress_bar=False)
        batch_emb = np.asarray(batch_emb, dtype=np.float32)
        embs.append(batch_emb)
    return np.vstack(embs)

def retrieve_top_k(question, all_ctx_emb, embedder, contexts, k=8):
    q_emb = embedder.encode([question], convert_to_numpy=True, device='cuda', normalize_embeddings=True, show_progress_bar=False)
    q_emb = np.asarray(q_emb, dtype=np.float32)
    sims = np.dot(all_ctx_emb, q_emb.T).squeeze(1)  # shape: (num_contexts,)
    topk_idx = np.argpartition(-sims, k)[:k]
    topk_idx = topk_idx[np.argsort(-sims[topk_idx])]
    hits = [contexts[i] for i in topk_idx if i < len(contexts)]
    return hits

# ------------------ 生成答案 ------------------

def generate_answer(question, contexts, tokenizer, model, gen_params):
    context_text = "\n".join(chunk_and_truncate_contexts(contexts))
    prompt = f"Question: {question}\nContext:\n{context_text}\nAnswer (if no answer, respond with 'no answer'):"

    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=512).to('cuda')

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=gen_params.get("max_new_tokens", 40),
            do_sample=gen_params.get("do_sample", False),
            num_beams=gen_params.get("num_beams", 2),
            no_repeat_ngram_size=gen_params.get("no_repeat_ngram_size", 3),
            early_stopping=gen_params.get("early_stopping", True),
        )
    answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
    if answer.startswith(prompt):
        answer = answer[len(prompt):]
    answer = answer.strip()
    if answer.lower() in ["false", "no answer", "none", ""]:
        answer = ""
    return answer

# ------------------ 主流程 ------------------

def main():
    train_file = '/kaggle/input/squad-train/train-v2.0.json'
    dev_file = '/kaggle/input/squad-dev/dev-v2.0.json'

    sentence_transformer_name = 'sentence-transformers/all-mpnet-base-v2'
    model_name = "google/flan-t5-xl"

    batch_size_embed = 128
    max_test = 500

    print("加载训练数据...")
    train_data = load_squad_dataset(train_file)
    print("加载验证数据...")
    dev_data = load_squad_dataset(dev_file)

    contexts = []
    for article in train_data:
        for para in article["paragraphs"]:
            contexts.append(para["context"])
    print(f"训练集上下文数量: {len(contexts)}")

    print("加载句向量模型（GPU）...")
    embedder = SentenceTransformer(sentence_transformer_name, device='cuda')

    print("计算所有上下文向量（GPU）...")
    all_ctx_emb = build_context_embeddings(contexts, embedder, batch_size=batch_size_embed)

    print(f"加载生成模型和tokenizer: {model_name}")
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSeq2SeqLM.from_pretrained(model_name).to('cuda')

    model.eval()
    preds = {}

    gen_params = {
        "max_new_tokens": 80,
        "num_beams": 5,
        "do_sample": False,
        "no_repeat_ngram_size": 3,
        "early_stopping": True,
    }

    print(f"开始对验证集前{max_test}个样本生成回答...")
    count = 0
    for article in dev_data:
        for para in article["paragraphs"]:
            for qa in para["qas"]:
                if count >= max_test:
                    break
                qid = qa['id']
                question = qa['question']

                retrieved = retrieve_top_k(question, all_ctx_emb, embedder, contexts, k=10)
                answer = generate_answer(question, retrieved, tokenizer, model, gen_params)

                preds[qid] = answer

                if count % 10 == 0:
                    print(f"已处理 {count} 个问题")
                count += 1
            if count >= max_test:
                break
        if count >= max_test:
            break

    filtered_dev_data = []
    for article in dev_data:
        new_article = {"paragraphs": []}
        for para in article['paragraphs']:
            new_para = {"qas": [], "context": para["context"]}
            for qa in para["qas"]:
                if qa['id'] in preds:
                    new_para["qas"].append(qa)
            if new_para["qas"]:
                new_article["paragraphs"].append(new_para)
        if new_article["paragraphs"]:
            filtered_dev_data.append(new_article)

    print("计算评测指标...")
    exact_scores, f1_scores = get_raw_scores(filtered_dev_data, preds)
    eval_results = make_eval_dict(exact_scores, f1_scores)

    print("评测结果：")
    for k, v in eval_results.items():
        print(f"{k}: {v:.2f}")

if __name__ == "__main__":
    main()


加载训练数据...
加载验证数据...
训练集上下文数量: 19035
加载句向量模型（GPU）...
计算所有上下文向量（GPU）...


Embedding contexts: 100%|██████████| 149/149 [05:00<00:00,  2.02s/it]
/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


加载生成模型和tokenizer: google/flan-t5-xl


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

开始对验证集前500个样本生成回答...
已处理 0 个问题
已处理 10 个问题
已处理 20 个问题
已处理 30 个问题
已处理 40 个问题
已处理 50 个问题
已处理 60 个问题
已处理 70 个问题
已处理 80 个问题
已处理 90 个问题
已处理 100 个问题
已处理 110 个问题
已处理 120 个问题
已处理 130 个问题
已处理 140 个问题
已处理 150 个问题
已处理 160 个问题
已处理 170 个问题
已处理 180 个问题
已处理 190 个问题
已处理 200 个问题
已处理 210 个问题
已处理 220 个问题
已处理 230 个问题
已处理 240 个问题
已处理 250 个问题
已处理 260 个问题
已处理 270 个问题
已处理 280 个问题
已处理 290 个问题
已处理 300 个问题
已处理 310 个问题
已处理 320 个问题
已处理 330 个问题
已处理 340 个问题
已处理 350 个问题
已处理 360 个问题
已处理 370 个问题
已处理 380 个问题
已处理 390 个问题
已处理 400 个问题
已处理 410 个问题
已处理 420 个问题
已处理 430 个问题
已处理 440 个问题
已处理 450 个问题
已处理 460 个问题
已处理 470 个问题
已处理 480 个问题
已处理 490 个问题
计算评测指标...
评测结果：
exact: 1.40
f1: 4.08
total: 500.00


xl加all mpnet base v2应该是最佳配置，在有限资源下

In [4]:
!pip install rank_bm25


##  BM25经典稀疏检索算法

In [1]:
import json
import collections
import re
import string
import numpy as np
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from sentence_transformers import SentenceTransformer
from tqdm import tqdm
from rank_bm25 import BM25Okapi

# ----------- 评测函数（与你原来相同） ---------------

def normalize_answer(s):
    if s is None:
        return ""
    def remove_articles(text):
        return re.sub(r'\b(a|an|the)\b', ' ', text)
    def white_space_fix(text):
        return ' '.join(text.split())
    def remove_punc(text):
        exclude = set(string.punctuation)
        return ''.join(ch for ch in text if ch not in exclude)
    def lower(text):
        return text.lower()
    return white_space_fix(remove_articles(remove_punc(lower(s))))

def get_tokens(s):
    if not s:
        return []
    return normalize_answer(s).split()

def compute_exact(a_gold, a_pred):
    return int(normalize_answer(a_gold) == normalize_answer(a_pred))

def compute_f1(a_gold, a_pred):
    gold_toks = get_tokens(a_gold)
    pred_toks = get_tokens(a_pred)
    common = collections.Counter(gold_toks) & collections.Counter(pred_toks)
    num_same = sum(common.values())
    if len(gold_toks) == 0 or len(pred_toks) == 0:
        return int(gold_toks == pred_toks)
    if num_same == 0:
        return 0
    precision = 1.0 * num_same / len(pred_toks)
    recall = 1.0 * num_same / len(gold_toks)
    return 2 * precision * recall / (precision + recall)

def get_raw_scores(dataset, preds):
    exact_scores = {}
    f1_scores = {}
    for article in dataset:
        for p in article['paragraphs']:
            for qa in p['qas']:
                qid = qa['id']
                gold_answers = [a['text'] for a in qa.get('answers', []) if normalize_answer(a.get('text', ''))]
                if not gold_answers:
                    gold_answers = ['']  # no answer questions
                a_pred = preds.get(qid, "")
                exact_scores[qid] = max(compute_exact(a, a_pred) for a in gold_answers)
                f1_scores[qid] = max(compute_f1(a, a_pred) for a in gold_answers)
    return exact_scores, f1_scores

def make_eval_dict(exact_scores, f1_scores):
    total = len(exact_scores)
    return collections.OrderedDict([
        ('exact', 100.0 * sum(exact_scores.values()) / total if total else 0),
        ('f1', 100.0 * sum(f1_scores.values()) / total if total else 0),
        ('total', total),
    ])

# ----------- 工具函数 -----------

def load_squad_dataset(path):
    with open(path, 'r', encoding='utf-8') as f:
        data_json = json.load(f)
    return data_json['data']

def tokenize_for_bm25(text):
    # 简单英文分词，BM25用
    return normalize_answer(text).split()

def preprocess_context(context, max_chars=1000):
    # 截断过长文本，防止显存爆炸
    return context if len(context) <= max_chars else context[:max_chars] + " ..."

# ----------- BM25构建 -----------

def build_bm25_index(contexts):
    tokenized_corpus = [tokenize_for_bm25(c) for c in contexts]
    bm25 = BM25Okapi(tokenized_corpus)
    return bm25, tokenized_corpus

# ----------- 向量检索 -----------

def embed_texts(texts, embedder, batch_size=16, device='cuda'):
    all_embs = []
    for i in range(0, len(texts), batch_size):
        batch = texts[i:i+batch_size]
        batch_emb = embedder.encode(batch, convert_to_numpy=True, device=device, normalize_embeddings=True, show_progress_bar=False)
        all_embs.append(batch_emb)
    return np.vstack(all_embs)

def retrieve_top_k(question, bm25, tokenized_corpus, contexts, embedder, vector_topk=10, bm25_topk=50, device='cuda'):
    # 1. BM25检索前bm25_topk条粗排
    q_tokens = tokenize_for_bm25(question)
    bm25_scores = bm25.get_scores(q_tokens)
    bm25_top_indices = np.argpartition(-bm25_scores, bm25_topk)[:bm25_topk]
    bm25_top_indices = bm25_top_indices[np.argsort(-bm25_scores[bm25_top_indices])]

    candidate_contexts = [contexts[i] for i in bm25_top_indices]
    candidate_tokenized = [tokenized_corpus[i] for i in bm25_top_indices]

    # 2. 向量编码 & 余弦相似度重新排序
    # 先对候选文本做长度截断
    candidate_contexts_trunc = [preprocess_context(c, max_chars=1000) for c in candidate_contexts]

    candidate_embs = embed_texts(candidate_contexts_trunc, embedder, batch_size=16, device=device)
    q_emb = embedder.encode([question], convert_to_numpy=True, device=device, normalize_embeddings=True)[0]

    sims = np.dot(candidate_embs, q_emb)
    topk_idx = np.argpartition(-sims, vector_topk)[:vector_topk]
    topk_idx = topk_idx[np.argsort(-sims[topk_idx])]

    # 返回最终topk文本
    results = [candidate_contexts_trunc[i] for i in topk_idx]
    return results

# ----------- 生成回答 -----------

def generate_answer(question, contexts, tokenizer, model, max_new_tokens=64):
    context_text = "\n".join(contexts)
    prompt = f"Question: {question}\nContext:\n{context_text}\nAnswer (if no answer, respond with 'no answer'):"

    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=512)
    device = next(model.parameters()).device
    inputs = {k: v.to(device) for k, v in inputs.items()}

    outputs = model.generate(
        **inputs,
        max_new_tokens=max_new_tokens,
        do_sample=False,
        num_beams=4,
        no_repeat_ngram_size=3,
        early_stopping=True
    )
    answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
    if answer.startswith(prompt):
        answer = answer[len(prompt):]
    answer = answer.strip()
    if answer.lower() in ["false", "no answer", "none", ""]:
        answer = ""
    return answer

# ----------- 主函数 -----------

def main():
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    train_file = '/kaggle/input/squad-train/train-v2.0.json'
    dev_file = '/kaggle/input/squad-dev/dev-v2.0.json'

    sentence_transformer_name = 'sentence-transformers/all-mpnet-base-v2'
    model_name = "google/flan-t5-xl"

    print("加载训练数据...")
    train_data = load_squad_dataset(train_file)
    print("加载验证数据...")
    dev_data = load_squad_dataset(dev_file)

    # 提取所有训练上下文
    contexts = []
    for article in train_data:
        for para in article["paragraphs"]:
            contexts.append(para["context"])
    print(f"训练集上下文数量: {len(contexts)}")

    print("加载句向量模型...")
    embedder = SentenceTransformer(sentence_transformer_name, device=device)

    print("构建BM25索引...")
    bm25, tokenized_corpus = build_bm25_index(contexts)

    print(f"加载生成模型和tokenizer: {model_name}")
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSeq2SeqLM.from_pretrained(model_name).to(device)
    model.eval()

    preds = {}
    max_test = 500
    count = 0

    print(f"开始对验证集前{max_test}个样本生成回答...")

    for article in dev_data:
        for para in article["paragraphs"]:
            for qa in para["qas"]:
                if count >= max_test:
                    break
                qid = qa['id']
                question = qa['question']

                retrieved = retrieve_top_k(question, bm25, tokenized_corpus, contexts, embedder,
                                         vector_topk=5, bm25_topk=50, device=device)
                answer = generate_answer(question, retrieved, tokenizer, model)

                preds[qid] = answer

                print(f"已处理样本数: {count+1}")
                count += 1
            if count >= max_test:
                break
        if count >= max_test:
            break

    # 构造只包含测试用的dev_data子集
    filtered_dev_data = []
    for article in dev_data:
        new_article = {"paragraphs": []}
        for para in article["paragraphs"]:
            new_para = {"qas": [], "context": para["context"]}
            for qa in para["qas"]:
                if qa['id'] in preds:
                    new_para["qas"].append(qa)
            if new_para["qas"]:
                new_article["paragraphs"].append(new_para)
        if new_article["paragraphs"]:
            filtered_dev_data.append(new_article)

    print("计算评测指标...")
    exact_scores, f1_scores = get_raw_scores(filtered_dev_data, preds)
    eval_results = make_eval_dict(exact_scores, f1_scores)

    print("评测结果：")
    for k, v in eval_results.items():
        print(f"{k}: {v:.2f}")

if __name__ == "__main__":
    main()


加载训练数据...
加载验证数据...
训练集上下文数量: 19035
加载句向量模型...
构建BM25索引...
加载生成模型和tokenizer: google/flan-t5-xl


/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

开始对验证集前500个样本生成回答...


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

已处理样本数: 498


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

已处理样本数: 499


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

已处理样本数: 500
计算评测指标...
评测结果：
exact: 2.20
f1: 4.89
total: 500.00


优化promt设计

In [ ]:
import json
import collections
import re
import string
import numpy as np
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from sentence_transformers import SentenceTransformer
from tqdm import tqdm

# ------------------ SQuAD Evaluation Functions ------------------

def normalize_answer(s):
    if s is None:
        return ""
    def remove_articles(text):
        return re.sub(r'\b(a|an|the)\b', ' ', text)
    def white_space_fix(text):
        return ' '.join(text.split())
    def remove_punc(text):
        exclude = set(string.punctuation)
        return ''.join(ch for ch in text if ch not in exclude)
    def lower(text):
        return text.lower()
    return white_space_fix(remove_articles(remove_punc(lower(s))))

def get_tokens(s):
    if not s:
        return []
    return normalize_answer(s).split()

def compute_exact(a_gold, a_pred):
    return int(normalize_answer(a_gold) == normalize_answer(a_pred))

def compute_f1(a_gold, a_pred):
    gold_toks = get_tokens(a_gold)
    pred_toks = get_tokens(a_pred)
    common = collections.Counter(gold_toks) & collections.Counter(pred_toks)
    num_same = sum(common.values())
    if len(gold_toks) == 0 or len(pred_toks) == 0:
        return int(gold_toks == pred_toks)
    if num_same == 0:
        return 0
    precision = 1.0 * num_same / len(pred_toks)
    recall = 1.0 * num_same / len(gold_toks)
    return 2 * precision * recall / (precision + recall)

def get_raw_scores(dataset, preds):
    exact_scores = {}
    f1_scores = {}
    for article in dataset:
        for p in article['paragraphs']:
            for qa in p['qas']:
                qid = qa['id']
                gold_answers = [a['text'] for a in qa.get('answers', []) if normalize_answer(a.get('text', ''))]
                if not gold_answers:
                    gold_answers = ['']  # no answer questions
                a_pred = preds.get(qid, "")
                exact_scores[qid] = max(compute_exact(a, a_pred) for a in gold_answers)
                f1_scores[qid] = max(compute_f1(a, a_pred) for a in gold_answers)
    return exact_scores, f1_scores

def make_eval_dict(exact_scores, f1_scores):
    total = len(exact_scores)
    return collections.OrderedDict([
        ('exact', 100.0 * sum(exact_scores.values()) / total if total else 0),
        ('f1', 100.0 * sum(f1_scores.values()) / total if total else 0),
        ('total', total),
    ])

# ------------------ RAG 主体流程 ------------------

def load_squad_dataset(path):
    with open(path, 'r', encoding='utf-8') as f:
        data_json = json.load(f)
    return data_json['data']

def chunk_and_truncate_contexts(contexts, max_chars=3000):
    out = []
    for c in contexts:
        if len(c) <= max_chars:
            out.append(c)
        else:
            out.append(c[:max_chars] + " ...")
    return out

def build_context_embeddings(contexts, embedder, batch_size=512):
    embs = []
    for i in tqdm(range(0, len(contexts), batch_size), desc="Embedding contexts"):
        batch = contexts[i:i+batch_size]
        batch_emb = embedder.encode(batch, convert_to_numpy=True, device='cuda', normalize_embeddings=True, show_progress_bar=False)
        batch_emb = np.asarray(batch_emb, dtype=np.float32)
        embs.append(batch_emb)
    embedder.to('cpu')  # 回收GPU显存
    return np.vstack(embs)

def retrieve_top_k(question, all_ctx_emb, embedder, contexts, k=20):
    q_emb = embedder.encode([question], convert_to_numpy=True, device='cuda', normalize_embeddings=True, show_progress_bar=False)
    q_emb = np.asarray(q_emb, dtype=np.float32)
    sims = np.dot(all_ctx_emb, q_emb.T).squeeze(1)
    topk_idx = np.argpartition(-sims, k)[:k]
    topk_idx = topk_idx[np.argsort(-sims[topk_idx])]
    hits = [contexts[i] for i in topk_idx if i < len(contexts)]
    return hits

def generate_answer_with_prompt(question, contexts, tokenizer, model, prompt_template, max_new_tokens=64):
    context_text = "\n".join(chunk_and_truncate_contexts(contexts))
    prompt = prompt_template.format(question=question, context=context_text)

    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=512)
    device = next(model.parameters()).device
    inputs = {k: v.to(device) for k, v in inputs.items()}

    outputs = model.generate(
        **inputs,
        max_new_tokens=max_new_tokens,
        do_sample=False,
        num_beams=4,
        no_repeat_ngram_size=3,
        early_stopping=True
    )
    answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
    if answer.startswith(prompt):
        answer = answer[len(prompt):]
    answer = answer.strip()
    if answer.lower() in ["false", "no answer", "none", ""]:
        answer = ""
    return answer

def main():
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    train_file = '/kaggle/input/squad-train/train-v2.0.json'
    dev_file = '/kaggle/input/squad-dev/dev-v2.0.json'

    sentence_transformer_name = 'sentence-transformers/all-mpnet-base-v2'
    model_name = "google/flan-t5-xl"
    batch_size_embed = 256
    max_test = 500

    print("加载训练数据...")
    train_data = load_squad_dataset(train_file)
    print("加载验证数据...")
    dev_data = load_squad_dataset(dev_file)

    contexts = []
    for article in train_data:
        for para in article["paragraphs"]:
            contexts.append(para["context"])
    print(f"训练集上下文数量: {len(contexts)}")

    print("加载句向量模型...")
    embedder = SentenceTransformer(sentence_transformer_name, device=device)

    print("GPU上计算所有上下文的向量表示...")
    all_ctx_emb = build_context_embeddings(contexts, embedder, batch_size=batch_size_embed)

    print(f"加载生成模型和tokenizer: {model_name}")
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSeq2SeqLM.from_pretrained(model_name).to(device)
    model.eval()

    # 定义多个Prompt模板，方便自动搜索最优
    prompt_templates = [
        "Question: {question}\nContext:\n{context}\nAnswer briefly. If no answer, say 'no answer'.",
        "Please answer the question based on the context below. If unknown, respond with 'no answer'.\nQuestion: {question}\nContext:\n{context}\nAnswer:",
        "Given the context, answer the question concisely. If not answerable, reply 'no answer'.\nQ: {question}\nContext:\n{context}\nA:",
        # 可以加更多模板
    ]

    best_prompt = None
    best_score = -1

    print(f"开始在验证集前{max_test}个样本上自动搜索最优Prompt...")
    for prompt_template in prompt_templates:
        preds = {}
        count = 0
        for article in dev_data:
            for para in article["paragraphs"]:
                for qa in para["qas"]:
                    if count >= max_test:
                        break
                    qid = qa['id']
                    question = qa['question']

                    retrieved = retrieve_top_k(question, all_ctx_emb, embedder, contexts, k=5)
                    answer = generate_answer_with_prompt(question, retrieved, tokenizer, model, prompt_template)

                    preds[qid] = answer
                    count += 1
                if count >= max_test:
                    break
            if count >= max_test:
                break

        # 过滤dev_data只保留测试样本对应的qa
        filtered_dev_data = []
        for article in dev_data:
            new_article = {"paragraphs": []}
            for para in article["paragraphs"]:
                new_para = {"qas": [], "context": para["context"]}
                for qa in para["qas"]:
                    if qa['id'] in preds:
                        new_para["qas"].append(qa)
                if new_para["qas"]:
                    new_article["paragraphs"].append(new_para)
            if new_article["paragraphs"]:
                filtered_dev_data.append(new_article)

        exact_scores, f1_scores = get_raw_scores(filtered_dev_data, preds)
        eval_results = make_eval_dict(exact_scores, f1_scores)

        print(f"Prompt模板: {prompt_template[:40]}... => Exact: {eval_results['exact']:.2f}, F1: {eval_results['f1']:.2f}")

        if eval_results['f1'] > best_score:
            best_score = eval_results['f1']
            best_prompt = prompt_template

    print(f"最优Prompt是:\n{best_prompt}")

    # 使用最优Prompt生成最终结果
    print(f"使用最优Prompt生成验证集回答...")
    final_preds = {}
    count = 0
    for article in dev_data:
        for para in article["paragraphs"]:
            for qa in para["qas"]:
                if count >= max_test:
                    break
                qid = qa['id']
                question = qa['question']

                retrieved = retrieve_top_k(question, all_ctx_emb, embedder, contexts, k=5)
                answer = generate_answer_with_prompt(question, retrieved, tokenizer, model, best_prompt)

                final_preds[qid] = answer
                count += 1
            if count >= max_test:
                break
        if count >= max_test:
            break

    filtered_dev_data = []
    for article in dev_data:
        new_article = {"paragraphs": []}
        for para in article["paragraphs"]:
            new_para = {"qas": [], "context": para["context"]}
            for qa in para["qas"]:
                if qa['id'] in final_preds:
                    new_para["qas"].append(qa)
            if new_para["qas"]:
                new_article["paragraphs"].append(new_para)
        if new_article["paragraphs"]:
            filtered_dev_data.append(new_article)

    exact_scores, f1_scores = get_raw_scores(filtered_dev_data, final_preds)
    eval_results = make_eval_dict(exact_scores, f1_scores)

    print("最终评测结果：")
    for k, v in eval_results.items():
        print(f"{k}: {v:.2f}")

if __name__ == "__main__":
    main()


500个qa 耗时约27h，不可接受，故作罢。

# 调节top k

In [2]:
import json
import collections
import re
import string
import numpy as np
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from sentence_transformers import SentenceTransformer
from tqdm import tqdm

# ------------------ SQuAD Evaluation Functions ------------------

def normalize_answer(s):
    if s is None:
        return ""
    def remove_articles(text):
        return re.sub(r'\b(a|an|the)\b', ' ', text)
    def white_space_fix(text):
        return ' '.join(text.split())
    def remove_punc(text):
        exclude = set(string.punctuation)
        return ''.join(ch for ch in text if ch not in exclude)
    def lower(text):
        return text.lower()
    return white_space_fix(remove_articles(remove_punc(lower(s))))

def get_tokens(s):
    if not s:
        return []
    return normalize_answer(s).split()

def compute_exact(a_gold, a_pred):
    return int(normalize_answer(a_gold) == normalize_answer(a_pred))

def compute_f1(a_gold, a_pred):
    gold_toks = get_tokens(a_gold)
    pred_toks = get_tokens(a_pred)
    common = collections.Counter(gold_toks) & collections.Counter(pred_toks)
    num_same = sum(common.values())
    if len(gold_toks) == 0 or len(pred_toks) == 0:
        return int(gold_toks == pred_toks)
    if num_same == 0:
        return 0
    precision = 1.0 * num_same / len(pred_toks)
    recall = 1.0 * num_same / len(gold_toks)
    return 2 * precision * recall / (precision + recall)

def get_raw_scores(dataset, preds):
    exact_scores = {}
    f1_scores = {}
    for article in dataset:
        for p in article['paragraphs']:
            for qa in p['qas']:
                qid = qa['id']
                gold_answers = [a['text'] for a in qa.get('answers', []) if normalize_answer(a.get('text', ''))]
                if not gold_answers:
                    gold_answers = ['']  # no answer questions
                a_pred = preds.get(qid, "")
                exact_scores[qid] = max(compute_exact(a, a_pred) for a in gold_answers)
                f1_scores[qid] = max(compute_f1(a, a_pred) for a in gold_answers)
    return exact_scores, f1_scores

def make_eval_dict(exact_scores, f1_scores):
    total = len(exact_scores)
    return collections.OrderedDict([
        ('exact', 100.0 * sum(exact_scores.values()) / total if total else 0),
        ('f1', 100.0 * sum(f1_scores.values()) / total if total else 0),
        ('total', total),
    ])

# ------------------ RAG 主体流程 ------------------

def load_squad_dataset(path):
    with open(path, 'r', encoding='utf-8') as f:
        data_json = json.load(f)
    return data_json['data']

def chunk_and_truncate_contexts(contexts, max_chars=3000):
    out = []
    for c in contexts:
        if len(c) <= max_chars:
            out.append(c)
        else:
            out.append(c[:max_chars] + " ...")
    return out

def build_context_embeddings(contexts, embedder, batch_size=256):
    embs = []
    for i in tqdm(range(0, len(contexts), batch_size), desc="Embedding contexts"):
        batch = contexts[i:i+batch_size]
        batch_emb = embedder.encode(batch, convert_to_numpy=True, device='cuda', normalize_embeddings=True, show_progress_bar=False)
        batch_emb = np.asarray(batch_emb, dtype=np.float32)
        embs.append(batch_emb)
    embedder.to('cpu')
    return np.vstack(embs)

def retrieve_top_k(question, all_ctx_emb, embedder, contexts, k=5, sim_threshold=None):
    q_emb = embedder.encode([question], convert_to_numpy=True, device='cuda', normalize_embeddings=True, show_progress_bar=False)
    q_emb = np.asarray(q_emb, dtype=np.float32)
    sims = np.dot(all_ctx_emb, q_emb.T).squeeze(1)
    
    if sim_threshold is not None:
        # 过滤相似度低的
        candidate_idxs = np.where(sims >= sim_threshold)[0]
        if len(candidate_idxs) == 0:
            # 没有高于阈值的，降级为无过滤
            candidate_idxs = np.arange(len(sims))
        sims_filtered = sims[candidate_idxs]
        topk_idx = candidate_idxs[np.argpartition(-sims_filtered, min(k,len(sims_filtered)-1))[:k]]
        topk_idx = topk_idx[np.argsort(-sims[topk_idx])]
    else:
        topk_idx = np.argpartition(-sims, k)[:k]
        topk_idx = topk_idx[np.argsort(-sims[topk_idx])]

    hits = [contexts[i] for i in topk_idx if i < len(contexts)]
    return hits

def generate_answer(question, contexts, tokenizer, model, max_new_tokens=64):
    context_text = "\n".join(chunk_and_truncate_contexts(contexts))
    prompt = f"Question: {question}\nContext:\n{context_text}\nAnswer (if no answer, respond with 'no answer'):"

    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=512)
    device = next(model.parameters()).device
    inputs = {k: v.to(device) for k, v in inputs.items()}

    outputs = model.generate(
        **inputs,
        max_new_tokens=max_new_tokens,
        do_sample=False,
        num_beams=4,
        no_repeat_ngram_size=3,
        early_stopping=True
    )
    answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
    if answer.startswith(prompt):
        answer = answer[len(prompt):]
    answer = answer.strip()
    if answer.lower() in ["false", "no answer", "none", ""]:
        answer = ""
    return answer

def evaluate_for_k(k, sim_threshold, dev_data, all_ctx_emb, embedder, contexts, tokenizer, model, max_test=500):
    preds = {}
    count = 0
    for article in dev_data:
        for para in article["paragraphs"]:
            for qa in para["qas"]:
                if count >= max_test:
                    break
                qid = qa["id"]
                question = qa["question"]

                retrieved = retrieve_top_k(question, all_ctx_emb, embedder, contexts, k=k, sim_threshold=sim_threshold)
                answer = generate_answer(question, retrieved, tokenizer, model)
                preds[qid] = answer

                count += 1
            if count >= max_test:
                break
        if count >= max_test:
            break

    filtered_dev_data = []
    for article in dev_data:
        new_article = {"paragraphs": []}
        for para in article["paragraphs"]:
            new_para = {"qas": [], "context": para["context"]}
            for qa in para["qas"]:
                if qa["id"] in preds:
                    new_para["qas"].append(qa)
            if new_para["qas"]:
                new_article["paragraphs"].append(new_para)
        if new_article["paragraphs"]:
            filtered_dev_data.append(new_article)

    exact_scores, f1_scores = get_raw_scores(filtered_dev_data, preds)
    eval_results = make_eval_dict(exact_scores, f1_scores)
    return eval_results

def main():
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    train_file = '/kaggle/input/squad-train/train-v2.0.json'
    dev_file = '/kaggle/input/squad-dev/dev-v2.0.json'

    sentence_transformer_name = 'sentence-transformers/all-mpnet-base-v2'
    model_name = "google/flan-t5-xl"
    batch_size_embed = 256

    print("加载训练数据...")
    train_data = load_squad_dataset(train_file)
    print("加载验证数据...")
    dev_data = load_squad_dataset(dev_file)

    contexts = []
    for article in train_data:
        for para in article["paragraphs"]:
            contexts.append(para["context"])
    print(f"训练集上下文数量: {len(contexts)}")

    print("加载句向量模型...")
    embedder = SentenceTransformer(sentence_transformer_name, device=device)

    print("GPU上计算所有上下文的向量表示...")
    all_ctx_emb = build_context_embeddings(contexts, embedder, batch_size=batch_size_embed)

    print(f"加载生成模型和tokenizer: {model_name}")
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSeq2SeqLM.from_pretrained(model_name).to(device)
    model.eval()

    # 尝试不同Top-K和相似度阈值组合
    top_k_list = [2,1]
    sim_threshold_list = [0.6, 0.5 , 0.4,0.3]

    best_f1 = 0
    best_params = None

    for k in top_k_list:
        for threshold in sim_threshold_list:
            print(f"评测参数: Top-K={k}, 相似度阈值={threshold}")
            results = evaluate_for_k(k, threshold, dev_data, all_ctx_emb, embedder, contexts, tokenizer, model)
            print(f"Exact: {results['exact']:.2f}, F1: {results['f1']:.2f}, Total: {results['total']}")
            if results['f1'] > best_f1:
                best_f1 = results['f1']
                best_params = (k, threshold)

    print(f"最佳参数 Top-K={best_params[0]}, 相似度阈值={best_params[1]}, F1={best_f1:.2f}")

if __name__ == "__main__":
    main()


加载训练数据...
加载验证数据...
训练集上下文数量: 19035
加载句向量模型...
GPU上计算所有上下文的向量表示...


Embedding contexts: 100%|██████████| 75/75 [04:14<00:00,  3.39s/it]


加载生成模型和tokenizer: google/flan-t5-xl


/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

评测参数: Top-K=2, 相似度阈值=0.6
Exact: 35.40, F1: 36.93, Total: 500
评测参数: Top-K=2, 相似度阈值=0.5
Exact: 35.60, F1: 37.13, Total: 500
评测参数: Top-K=2, 相似度阈值=0.4
Exact: 35.80, F1: 37.46, Total: 500
评测参数: Top-K=2, 相似度阈值=0.3
Exact: 34.80, F1: 36.46, Total: 500
评测参数: Top-K=1, 相似度阈值=0.6
Exact: 44.60, F1: 45.79, Total: 500
评测参数: Top-K=1, 相似度阈值=0.5
Exact: 44.60, F1: 45.79, Total: 500
评测参数: Top-K=1, 相似度阈值=0.4
Exact: 44.60, F1: 45.79, Total: 500
评测参数: Top-K=1, 相似度阈值=0.3
Exact: 44.60, F1: 45.79, Total: 500
最佳参数 Top-K=1, 相似度阈值=0.6, F1=45.79


从4分到20分，单靠阈值调节几乎不可能实现，故大概率是k往小，阈值往小

评测参数: Top-K=3, 相似度阈值=None
Exact: 18.40, F1: 20.44, Total: 500
评测参数: Top-K=3, 相似度阈值=0.6
Exact: 19.60, F1: 21.64, Total: 500
评测参数: Top-K=3, 相似度阈值=0.7

如果把k往大了调，成绩下降

评测参数: Top-K=1, 相似度阈值=0.2
Exact: 44.60, F1: 45.79, Total: 500

BM25融合向量相似度增强检索相关度 上下文先分句，再按字符数（1500）截断，减少噪声
## BM25 + 向量检索融合（hybrid_search）

top n = 3 ,beam = 1,max token  = 32

In [1]:
import json
import collections
import re
import string
import numpy as np
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from sentence_transformers import SentenceTransformer
from tqdm import tqdm
from rank_bm25 import BM25Okapi

# --------- SQuAD Evaluation Functions ---------

def normalize_answer(s):
    if s is None:
        return ""
    def remove_articles(text):
        return re.sub(r'\b(a|an|the)\b', ' ', text)
    def white_space_fix(text):
        return ' '.join(text.split())
    def remove_punc(text):
        exclude = set(string.punctuation)
        return ''.join(ch for ch in text if ch not in exclude)
    def lower(text):
        return text.lower()
    return white_space_fix(remove_articles(remove_punc(lower(s))))

def get_tokens(s):
    if not s:
        return []
    return normalize_answer(s).split()

def compute_exact(a_gold, a_pred):
    return int(normalize_answer(a_gold) == normalize_answer(a_pred))

def compute_f1(a_gold, a_pred):
    gold_toks = get_tokens(a_gold)
    pred_toks = get_tokens(a_pred)
    common = collections.Counter(gold_toks) & collections.Counter(pred_toks)
    num_same = sum(common.values())
    if len(gold_toks) == 0 or len(pred_toks) == 0:
        return int(gold_toks == pred_toks)
    if num_same == 0:
        return 0
    precision = 1.0 * num_same / len(pred_toks)
    recall = 1.0 * num_same / len(gold_toks)
    return 2 * precision * recall / (precision + recall)

def get_raw_scores(dataset, preds):
    exact_scores = {}
    f1_scores = {}
    for article in dataset:
        for p in article['paragraphs']:
            for qa in p['qas']:
                qid = qa['id']
                gold_answers = [a['text'] for a in qa.get('answers', []) if normalize_answer(a.get('text', ''))]
                if not gold_answers:
                    gold_answers = ['']  # no answer questions
                a_pred = preds.get(qid, "")
                exact_scores[qid] = max(compute_exact(a, a_pred) for a in gold_answers)
                f1_scores[qid] = max(compute_f1(a, a_pred) for a in gold_answers)
    return exact_scores, f1_scores

def make_eval_dict(exact_scores, f1_scores):
    total = len(exact_scores)
    return collections.OrderedDict([
        ('exact', 100.0 * sum(exact_scores.values()) / total if total else 0),
        ('f1', 100.0 * sum(f1_scores.values()) / total if total else 0),
        ('total', total),
    ])

# --------- 上下文处理和检索 ---------

def build_bm25_index(contexts):
    tokenized_corpus = [ctx.split() for ctx in contexts]
    bm25 = BM25Okapi(tokenized_corpus)
    return bm25

def hybrid_search(question, bm25, embedder, contexts, all_ctx_emb, k=1, bm25_top_n=5, sim_threshold=0.3):
    tokenized_query = question.split()
    bm25_scores = bm25.get_scores(tokenized_query)
    bm25_top_idx = np.argsort(bm25_scores)[::-1][:bm25_top_n]
    candidate_contexts = [contexts[i] for i in bm25_top_idx]
    candidate_embs = all_ctx_emb[bm25_top_idx]

    q_emb = embedder.encode([question], convert_to_numpy=True, device='cuda', normalize_embeddings=True, show_progress_bar=False)
    q_emb = np.asarray(q_emb, dtype=np.float32)

    sims = np.dot(candidate_embs, q_emb.T).squeeze(1)

    filtered_idxs = np.where(sims >= sim_threshold)[0]
    if len(filtered_idxs) == 0:
        filtered_idxs = np.arange(len(sims))

    filtered_sims = sims[filtered_idxs]
    filtered_bm25_idx = bm25_top_idx[filtered_idxs]

    bm25_scores_filtered = np.array(bm25_scores)[filtered_bm25_idx]

    # 归一化分数
    bm25_norm = (bm25_scores_filtered - bm25_scores_filtered.min()) / (bm25_scores_filtered.max() - bm25_scores_filtered.min() + 1e-8)
    sim_norm = (filtered_sims - filtered_sims.min()) / (filtered_sims.max() - filtered_sims.min() + 1e-8)

    combined_scores = 0.5 * bm25_norm + 0.5 * sim_norm

    topk_idx = np.argsort(combined_scores)[::-1][:k]
    selected_contexts = [contexts[filtered_bm25_idx[i]] for i in topk_idx]
    return selected_contexts

def generate_answer(question, contexts, tokenizer, model):
    device = next(model.parameters()).device
    context_text = "\n".join(contexts)
    prompt = f"Question: {question}\nContext:\n{context_text}\nAnswer (if no answer, respond with 'no answer'):"

    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=512)
    inputs = {k: v.to(device) for k, v in inputs.items()}

    outputs = model.generate(
        **inputs,
        max_new_tokens=16,
        num_beams=1,
        do_sample=False,
        no_repeat_ngram_size=3,
        early_stopping=True
    )
    answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
    if answer.startswith(prompt):
        answer = answer[len(prompt):]
    answer = answer.strip()
    if answer.lower() in ["false", "no answer", "none", ""]:
        answer = ""
    return answer

# --------- 主流程 ---------

def load_squad_dataset(path):
    with open(path, 'r', encoding='utf-8') as f:
        data_json = json.load(f)
    return data_json['data']

def main():
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    train_file = '/kaggle/input/squad-train/train-v2.0.json'
    dev_file = '/kaggle/input/squad-dev/dev-v2.0.json'

    sentence_transformer_name = 'sentence-transformers/all-mpnet-base-v2'
    model_name = "google/flan-t5-xl"

    print("加载训练数据...")
    train_data = load_squad_dataset(train_file)
    print("加载验证数据...")
    dev_data = load_squad_dataset(dev_file)

    print("提取训练集上下文...")
    contexts = []
    for article in train_data:
        for para in article["paragraphs"]:
            contexts.append(para["context"])

    print(f"训练集上下文数量: {len(contexts)}")

    print("加载句向量模型...")
    embedder = SentenceTransformer(sentence_transformer_name, device=device)

    print("GPU上计算所有上下文向量...")
    batch_size_embed = 256
    all_ctx_emb = []
    for i in tqdm(range(0, len(contexts), batch_size_embed), desc="Embedding contexts"):
        batch = contexts[i:i+batch_size_embed]
        batch_emb = embedder.encode(batch, convert_to_numpy=True, device='cuda', normalize_embeddings=True, show_progress_bar=False)
        all_ctx_emb.append(batch_emb)
    all_ctx_emb = np.vstack(all_ctx_emb)

    print("构建BM25索引...")
    bm25 = build_bm25_index(contexts)

    print(f"加载生成模型和tokenizer: {model_name}")
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSeq2SeqLM.from_pretrained(model_name).to(device)
    model.eval()

    k = 1
    sim_threshold = 0.3
    bm25_top_n = 5
    max_test = 500

    preds = {}
    count = 0
    print("开始评测...")
    for article in tqdm(dev_data, desc="Eval articles"):
        for para in article["paragraphs"]:
            for qa in para["qas"]:
                if count >= max_test:
                    break
                qid = qa["id"]
                question = qa["question"]

                retrieved_contexts = hybrid_search(question, bm25, embedder, contexts, all_ctx_emb, k=k, bm25_top_n=bm25_top_n, sim_threshold=sim_threshold)
                answer = generate_answer(question, retrieved_contexts, tokenizer, model)
                preds[qid] = answer

                count += 1
            if count >= max_test:
                break
        if count >= max_test:
            break

    # 过滤验证集，保留预测题目
    filtered_dev_data = []
    for article in dev_data:
        new_article = {"paragraphs": []}
        for para in article["paragraphs"]:
            new_para = {"qas": [], "context": para["context"]}
            for qa in para["qas"]:
                if qa["id"] in preds:
                    new_para["qas"].append(qa)
            if new_para["qas"]:
                new_article["paragraphs"].append(new_para)
        if new_article["paragraphs"]:
            filtered_dev_data.append(new_article)

    exact_scores, f1_scores = get_raw_scores(filtered_dev_data, preds)
    eval_results = make_eval_dict(exact_scores, f1_scores)

    print(f"评测结果: Exact={eval_results['exact']:.2f}, F1={eval_results['f1']:.2f}, 总样本数={eval_results['total']}")

if __name__ == "__main__":
    main()


加载训练数据...
加载验证数据...
提取训练集上下文...
训练集上下文数量: 19035
加载句向量模型...
GPU上计算所有上下文向量...


Embedding contexts: 100%|██████████| 75/75 [04:59<00:00,  3.99s/it]


构建BM25索引...
加载生成模型和tokenizer: google/flan-t5-xl


/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

开始评测...


Eval articles:   0%|          | 0/35 [00:00<?, ?it/s]/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:433: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(
Eval articles:   3%|▎         | 1/35 [04:05<2:19:03, 245.41s/it]

评测结果: Exact=46.60, F1=46.94, 总样本数=500


大轮测

In [ ]:
import json
import collections
import re
import string
import numpy as np
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from sentence_transformers import SentenceTransformer
from tqdm import tqdm
from rank_bm25 import BM25Okapi
import nltk

nltk.download('punkt')
from nltk.tokenize import sent_tokenize

# --------- SQuAD Evaluation Functions ---------

def normalize_answer(s):
    if s is None:
        return ""
    def remove_articles(text):
        return re.sub(r'\b(a|an|the)\b', ' ', text)
    def white_space_fix(text):
        return ' '.join(text.split())
    def remove_punc(text):
        exclude = set(string.punctuation)
        return ''.join(ch for ch in text if ch not in exclude)
    def lower(text):
        return text.lower()
    return white_space_fix(remove_articles(remove_punc(lower(s))))

def get_tokens(s):
    if not s:
        return []
    return normalize_answer(s).split()

def compute_exact(a_gold, a_pred):
    return int(normalize_answer(a_gold) == normalize_answer(a_pred))

def compute_f1(a_gold, a_pred):
    gold_toks = get_tokens(a_gold)
    pred_toks = get_tokens(a_pred)
    common = collections.Counter(gold_toks) & collections.Counter(pred_toks)
    num_same = sum(common.values())
    if len(gold_toks) == 0 or len(pred_toks) == 0:
        return int(gold_toks == pred_toks)
    if num_same == 0:
        return 0
    precision = 1.0 * num_same / len(pred_toks)
    recall = 1.0 * num_same / len(gold_toks)
    return 2 * precision * recall / (precision + recall)

def get_raw_scores(dataset, preds):
    exact_scores = {}
    f1_scores = {}
    for article in dataset:
        for p in article['paragraphs']:
            for qa in p['qas']:
                qid = qa['id']
                gold_answers = [a['text'] for a in qa.get('answers', []) if normalize_answer(a.get('text', ''))]
                if not gold_answers:
                    gold_answers = ['']  # no answer questions
                a_pred = preds.get(qid, "")
                exact_scores[qid] = max(compute_exact(a, a_pred) for a in gold_answers)
                f1_scores[qid] = max(compute_f1(a, a_pred) for a in gold_answers)
    return exact_scores, f1_scores

def make_eval_dict(exact_scores, f1_scores):
    total = len(exact_scores)
    return collections.OrderedDict([
        ('exact', 100.0 * sum(exact_scores.values()) / total if total else 0),
        ('f1', 100.0 * sum(f1_scores.values()) / total if total else 0),
        ('total', total),
    ])

# --------- 上下文拆分和检索 ---------

def chunk_context(text, max_tokens=100):
    # 简单按句子拆分，且合并成块，每块近似max_tokens词
    sentences = sent_tokenize(text)
    chunks = []
    current_chunk = []
    current_len = 0
    for sent in sentences:
        sent_len = len(sent.split())
        if current_len + sent_len > max_tokens and current_chunk:
            chunks.append(' '.join(current_chunk))
            current_chunk = [sent]
            current_len = sent_len
        else:
            current_chunk.append(sent)
            current_len += sent_len
    if current_chunk:
        chunks.append(' '.join(current_chunk))
    return chunks

def build_bm25_index(context_chunks):
    tokenized_corpus = [ctx.split() for ctx in context_chunks]
    bm25 = BM25Okapi(tokenized_corpus)
    return bm25

def hybrid_search(question, bm25, embedder, context_chunks, all_ctx_emb,
                  k=1, bm25_top_n=5, sim_threshold=0.3):
    tokenized_query = question.split()
    bm25_scores = bm25.get_scores(tokenized_query)
    bm25_top_idx = np.argsort(bm25_scores)[::-1][:bm25_top_n]
    candidate_contexts = [context_chunks[i] for i in bm25_top_idx]
    candidate_embs = all_ctx_emb[bm25_top_idx]

    q_emb = embedder.encode([question], convert_to_numpy=True, device='cuda', normalize_embeddings=True, show_progress_bar=False)
    q_emb = np.asarray(q_emb, dtype=np.float32)

    sims = np.dot(candidate_embs, q_emb.T).squeeze(1)

    filtered_idxs = np.where(sims >= sim_threshold)[0]
    if len(filtered_idxs) == 0:
        filtered_idxs = np.arange(len(sims))

    filtered_sims = sims[filtered_idxs]
    filtered_bm25_idx = bm25_top_idx[filtered_idxs]

    bm25_scores_filtered = np.array(bm25_scores)[filtered_bm25_idx]

    bm25_norm = (bm25_scores_filtered - bm25_scores_filtered.min()) / (bm25_scores_filtered.max() - bm25_scores_filtered.min() + 1e-8)
    sim_norm = (filtered_sims - filtered_sims.min()) / (filtered_sims.max() - filtered_sims.min() + 1e-8)

    combined_scores = 0.5 * bm25_norm + 0.5 * sim_norm

    topk_idx = np.argsort(combined_scores)[::-1][:k]
    selected_contexts = [context_chunks[filtered_bm25_idx[i]] for i in topk_idx]
    return selected_contexts

def generate_answer(question, contexts, tokenizer, model, max_new_tokens=16, num_beams=1):
    device = next(model.parameters()).device
    context_text = "\n".join(contexts)
    prompt = f"Question: {question}\nContext:\n{context_text}\nAnswer (if no answer, respond with 'no answer'):"

    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=512)
    inputs = {k: v.to(device) for k, v in inputs.items()}

    outputs = model.generate(
        **inputs,
        max_new_tokens=max_new_tokens,
        num_beams=num_beams,
        do_sample=False,
        no_repeat_ngram_size=3,
        early_stopping=True
    )
    answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
    if answer.startswith(prompt):
        answer = answer[len(prompt):]
    answer = answer.strip()
    if answer.lower() in ["false", "no answer", "none", ""]:
        answer = ""
    return answer

def load_squad_dataset(path):
    with open(path, 'r', encoding='utf-8') as f:
        data_json = json.load(f)
    return data_json['data']

def run_evaluation(dev_data, bm25, embedder, context_chunks, all_ctx_emb, tokenizer, model,
                   k=1, sim_threshold=0.3,
                   bm25_top_n=5, num_beams=1, max_new_tokens=16, max_test=500):

    preds = {}
    count = 0
    total_qas = sum(len(para["qas"]) for article in dev_data for para in article["paragraphs"])
    max_test = min(max_test, total_qas)

    with tqdm(total=max_test, desc=f"Eval QAs top_n={bm25_top_n} beam={num_beams} tokens={max_new_tokens}") as pbar:
        for article in dev_data:
            for para in article["paragraphs"]:
                for qa in para["qas"]:
                    if count >= max_test:
                        break
                    qid = qa["id"]
                    question = qa["question"]

                    retrieved_contexts = hybrid_search(
                        question, bm25, embedder, context_chunks, all_ctx_emb,
                        k=k, bm25_top_n=bm25_top_n, sim_threshold=sim_threshold)
                    answer = generate_answer(
                        question, retrieved_contexts, tokenizer, model,
                        max_new_tokens=max_new_tokens, num_beams=num_beams)
                    preds[qid] = answer

                    count += 1
                    pbar.update(1)
                if count >= max_test:
                    break
            if count >= max_test:
                break

    # 过滤验证集，保留预测题目
    filtered_dev_data = []
    for article in dev_data:
        new_article = {"paragraphs": []}
        for para in article["paragraphs"]:
            new_para = {"qas": [], "context": para["context"]}
            for qa in para["qas"]:
                if qa["id"] in preds:
                    new_para["qas"].append(qa)
            if new_para["qas"]:
                new_article["paragraphs"].append(new_para)
        if new_article["paragraphs"]:
            filtered_dev_data.append(new_article)

    exact_scores, f1_scores = get_raw_scores(filtered_dev_data, preds)
    eval_results = make_eval_dict(exact_scores, f1_scores)
    return eval_results

def main():
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    train_file = '/kaggle/input/squad-train/train-v2.0.json'
    dev_file = '/kaggle/input/squad-dev/dev-v2.0.json'

    sentence_transformer_name = 'sentence-transformers/all-mpnet-base-v2'
    model_name = "google/flan-t5-xl"

    print("加载训练数据...")
    train_data = load_squad_dataset(train_file)
    print("加载验证数据...")
    dev_data = load_squad_dataset(dev_file)

    print("拆分上下文为chunk...")
    context_chunks = []
    for article in train_data:
        for para in article["paragraphs"]:
            chunks = chunk_context(para["context"], max_tokens=100)  # 每chunk约100词
            context_chunks.extend(chunks)
    print(f"拆分后上下文chunk数量: {len(context_chunks)}")

    print("加载句向量模型...")
    embedder = SentenceTransformer(sentence_transformer_name, device=device)

    print("GPU上计算所有chunk向量...")
    batch_size_embed = 256
    all_ctx_emb = []
    for i in tqdm(range(0, len(context_chunks), batch_size_embed), desc="Embedding chunks"):
        batch = context_chunks[i:i+batch_size_embed]
        batch_emb = embedder.encode(batch, convert_to_numpy=True, device=device, normalize_embeddings=True, show_progress_bar=False)
        all_ctx_emb.append(batch_emb)
    all_ctx_emb = np.vstack(all_ctx_emb)

    print("构建BM25索引...")
    bm25 = build_bm25_index(context_chunks)

    print(f"加载生成模型和tokenizer: {model_name}")
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSeq2SeqLM.from_pretrained(model_name).to(device)
    model.eval()

    # 大轮测参数空间
    param_grid = {
        'bm25_top_n': [4, 6],
        'num_beams': [2, 3],
        'max_new_tokens': [64, 16],
    }

    k = 1
    sim_threshold = 0.3
    max_test = 500

    for top_n in param_grid['bm25_top_n']:
        for beam in param_grid['num_beams']:
            for max_tokens in param_grid['max_new_tokens']:
                print(f"\n===== 测试参数组合 top_n={top_n}, beam={beam}, max_tokens={max_tokens} =====")
                results = run_evaluation(
                    dev_data, bm25, embedder, context_chunks, all_ctx_emb, tokenizer, model,
                    k=k,
                    sim_threshold=sim_threshold,
                    bm25_top_n=top_n,
                    num_beams=beam,
                    max_new_tokens=max_tokens,
                    max_test=max_test
                )
                print(f"结果: Exact={results['exact']:.2f}, F1={results['f1']:.2f}, 测试样本数={results['total']}")

if __name__ == "__main__":
    main()


[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


加载训练数据...
加载验证数据...
拆分上下文为chunk...
拆分后上下文chunk数量: 32102
加载句向量模型...
GPU上计算所有chunk向量...


Embedding chunks: 100%|██████████| 126/126 [04:56<00:00,  2.35s/it]


构建BM25索引...
加载生成模型和tokenizer: google/flan-t5-xl


/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


...

Eval QAs top_n=4 beam=2 tokens=64: 100%|██████████| 500/500 [04:34<00:00,  1.82it/s]


结果: Exact=48.20, F1=48.50, 测试样本数=500

===== 测试参数组合 top_n=4, beam=2, max_tokens=16 =====


Eval QAs top_n=4 beam=2 tokens=16: 100%|██████████| 500/500 [04:34<00:00,  1.82it/s]


结果: Exact=48.20, F1=48.50, 测试样本数=500

===== 测试参数组合 top_n=4, beam=3, max_tokens=64 =====


Eval QAs top_n=4 beam=3 tokens=64:  86%|████████▋ | 432/500 [04:21<00:42,  1.61it/s]

   top_n = 4
    ，beam = 6
    ，max_tokens = 16
    ，k = 1
    ，sim_threshold = 0.3
    ，max_test = 500

调节max char

In [ ]:


import json
import collections
import re
import string
import numpy as np
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from sentence_transformers import SentenceTransformer
from tqdm import tqdm
from rank_bm25 import BM25Okapi
import nltk

nltk.download('punkt')
from nltk.tokenize import sent_tokenize

# --------- SQuAD Evaluation Functions ---------

def normalize_answer(s):
    if s is None:
        return ""
    def remove_articles(text):
        return re.sub(r'\b(a|an|the)\b', ' ', text)
    def white_space_fix(text):
        return ' '.join(text.split())
    def remove_punc(text):
        exclude = set(string.punctuation)
        return ''.join(ch for ch in text if ch not in exclude)
    def lower(text):
        return text.lower()
    return white_space_fix(remove_articles(remove_punc(lower(s))))

def get_tokens(s):
    if not s:
        return []
    return normalize_answer(s).split()

def compute_exact(a_gold, a_pred):
    return int(normalize_answer(a_gold) == normalize_answer(a_pred))

def compute_f1(a_gold, a_pred):
    gold_toks = get_tokens(a_gold)
    pred_toks = get_tokens(a_pred)
    common = collections.Counter(gold_toks) & collections.Counter(pred_toks)
    num_same = sum(common.values())
    if len(gold_toks) == 0 or len(pred_toks) == 0:
        return int(gold_toks == pred_toks)
    if num_same == 0:
        return 0
    precision = 1.0 * num_same / len(pred_toks)
    recall = 1.0 * num_same / len(gold_toks)
    return 2 * precision * recall / (precision + recall)

def get_raw_scores(dataset, preds):
    exact_scores = {}
    f1_scores = {}
    for article in dataset:
        for p in article['paragraphs']:
            for qa in p['qas']:
                qid = qa['id']
                gold_answers = [a['text'] for a in qa.get('answers', []) if normalize_answer(a.get('text', ''))]
                if not gold_answers:
                    gold_answers = ['']  # no answer questions
                a_pred = preds.get(qid, "")
                exact_scores[qid] = max(compute_exact(a, a_pred) for a in gold_answers)
                f1_scores[qid] = max(compute_f1(a, a_pred) for a in gold_answers)
    return exact_scores, f1_scores

def make_eval_dict(exact_scores, f1_scores):
    total = len(exact_scores)
    return collections.OrderedDict([
        ('exact', 100.0 * sum(exact_scores.values()) / total if total else 0),
        ('f1', 100.0 * sum(f1_scores.values()) / total if total else 0),
        ('total', total),
    ])

# --------- 上下文拆分和检索 ---------

def chunk_context(text, max_tokens=100):
    sentences = sent_tokenize(text)
    chunks = []
    current_chunk = []
    current_len = 0
    for sent in sentences:
        sent_len = len(sent.split())
        if current_len + sent_len > max_tokens and current_chunk:
            chunks.append(' '.join(current_chunk))
            current_chunk = [sent]
            current_len = sent_len
        else:
            current_chunk.append(sent)
            current_len += sent_len
    if current_chunk:
        chunks.append(' '.join(current_chunk))
    return chunks

def build_bm25_index(context_chunks):
    tokenized_corpus = [ctx.split() for ctx in context_chunks]
    bm25 = BM25Okapi(tokenized_corpus)
    return bm25

def hybrid_search(question, bm25, embedder, context_chunks, all_ctx_emb,
                  k=1, bm25_top_n=5, sim_threshold=0.3):
    tokenized_query = question.split()
    bm25_scores = bm25.get_scores(tokenized_query)
    bm25_top_idx = np.argsort(bm25_scores)[::-1][:bm25_top_n]
    candidate_contexts = [context_chunks[i] for i in bm25_top_idx]
    candidate_embs = all_ctx_emb[bm25_top_idx]

    q_emb = embedder.encode([question], convert_to_numpy=True, device='cuda', normalize_embeddings=True, show_progress_bar=False)
    q_emb = np.asarray(q_emb, dtype=np.float32)

    sims = np.dot(candidate_embs, q_emb.T).squeeze(1)

    filtered_idxs = np.where(sims >= sim_threshold)[0]
    if len(filtered_idxs) == 0:
        filtered_idxs = np.arange(len(sims))

    filtered_sims = sims[filtered_idxs]
    filtered_bm25_idx = bm25_top_idx[filtered_idxs]

    bm25_scores_filtered = np.array(bm25_scores)[filtered_bm25_idx]

    bm25_norm = (bm25_scores_filtered - bm25_scores_filtered.min()) / (bm25_scores_filtered.max() - bm25_scores_filtered.min() + 1e-8)
    sim_norm = (filtered_sims - filtered_sims.min()) / (filtered_sims.max() - filtered_sims.min() + 1e-8)

    combined_scores = 0.5 * bm25_norm + 0.5 * sim_norm

    topk_idx = np.argsort(combined_scores)[::-1][:k]
    selected_contexts = [context_chunks[filtered_bm25_idx[i]] for i in topk_idx]
    return selected_contexts

def generate_answer(question, contexts, tokenizer, model, max_new_tokens=16, num_beams=1, max_context_chars=1500):
    device = next(model.parameters()).device

    total_chars = 0
    limited_contexts = []
    for c in contexts:
        if total_chars + len(c) > max_context_chars:
            break
        limited_contexts.append(c)
        total_chars += len(c)

    context_text = "\n".join(limited_contexts)
    prompt = f"Question: {question}\nContext:\n{context_text}\nAnswer (if no answer, respond with 'no answer'):"

    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=512)
    inputs = {k: v.to(device) for k, v in inputs.items()}

    outputs = model.generate(
        **inputs,
        max_new_tokens=max_new_tokens,
        num_beams=num_beams,
        do_sample=False,
        no_repeat_ngram_size=3,
        early_stopping=True
    )
    answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
    if answer.startswith(prompt):
        answer = answer[len(prompt):]
    answer = answer.strip()
    if answer.lower() in ["false", "no answer", "none", ""]:
        answer = ""
    return answer

def load_squad_dataset(path):
    with open(path, 'r', encoding='utf-8') as f:
        data_json = json.load(f)
    return data_json['data']

def run_evaluation(dev_data, bm25, embedder, context_chunks, all_ctx_emb, tokenizer, model,
                   k=1, sim_threshold=0.3,
                   bm25_top_n=4, num_beams=4, max_new_tokens=16,
                   max_context_chars=1500,
                   max_test=500):

    preds = {}
    count = 0
    total_qas = sum(len(para["qas"]) for article in dev_data for para in article["paragraphs"])
    max_test = min(max_test, total_qas)

    with tqdm(total=max_test, desc=f"Eval QAs top_n={bm25_top_n} beam={num_beams} tokens={max_new_tokens} max_chars={max_context_chars}") as pbar:
        for article in dev_data:
            for para in article["paragraphs"]:
                for qa in para["qas"]:
                    if count >= max_test:
                        break
                    qid = qa["id"]
                    question = qa["question"]

                    retrieved_contexts = hybrid_search(
                        question, bm25, embedder, context_chunks, all_ctx_emb,
                        k=k, bm25_top_n=bm25_top_n, sim_threshold=sim_threshold)
                    answer = generate_answer(
                        question, retrieved_contexts, tokenizer, model,
                        max_new_tokens=max_new_tokens, num_beams=num_beams,
                        max_context_chars=max_context_chars)
                    preds[qid] = answer

                    count += 1
                    pbar.update(1)
                if count >= max_test:
                    break
            if count >= max_test:
                break

    filtered_dev_data = []
    for article in dev_data:
        new_article = {"paragraphs": []}
        for para in article["paragraphs"]:
            new_para = {"qas": [], "context": para["context"]}
            for qa in para["qas"]:
                if qa["id"] in preds:
                    new_para["qas"].append(qa)
            if new_para["qas"]:
                new_article["paragraphs"].append(new_para)
        if new_article["paragraphs"]:
            filtered_dev_data.append(new_article)

    exact_scores, f1_scores = get_raw_scores(filtered_dev_data, preds)
    eval_results = make_eval_dict(exact_scores, f1_scores)
    return eval_results

def main():
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    train_file = '/kaggle/input/squad-train/train-v2.0.json'
    dev_file = '/kaggle/input/squad-dev/dev-v2.0.json'

    sentence_transformer_name = 'sentence-transformers/all-mpnet-base-v2'
    model_name = "google/flan-t5-xl"

    print("加载训练数据...")
    train_data = load_squad_dataset(train_file)
    print("加载验证数据...")
    dev_data = load_squad_dataset(dev_file)

    print("拆分上下文为chunk...")
    context_chunks = []
    for article in train_data:
        for para in article["paragraphs"]:
            chunks = chunk_context(para["context"], max_tokens=100)
            context_chunks.extend(chunks)
    print(f"拆分后上下文chunk数量: {len(context_chunks)}")

    print("加载句向量模型...")
    embedder = SentenceTransformer(sentence_transformer_name, device=device)

    print("GPU上计算所有chunk向量...")
    batch_size_embed = 256
    all_ctx_emb = []
    for i in tqdm(range(0, len(context_chunks), batch_size_embed), desc="Embedding chunks"):
        batch = context_chunks[i:i+batch_size_embed]
        batch_emb = embedder.encode(batch, convert_to_numpy=True, device=device, normalize_embeddings=True, show_progress_bar=False)
        all_ctx_emb.append(batch_emb)
    all_ctx_emb = np.vstack(all_ctx_emb)

    print("构建BM25索引...")
    bm25 = build_bm25_index(context_chunks)

    print(f"加载生成模型和tokenizer: {model_name}")
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSeq2SeqLM.from_pretrained(model_name).to(device)
    model.eval()

    max_context_chars_list = [5,10,20,30]

    for max_chars in max_context_chars_list:
        print(f"\n===== 测试 max_context_chars = {max_chars} =====")
        results = run_evaluation(
            dev_data, bm25, embedder, context_chunks, all_ctx_emb, tokenizer, model,
            k=1,
            sim_threshold=0.3,
            bm25_top_n=4,
            num_beams=8,
            max_new_tokens=16,
            max_context_chars=max_chars,
            max_test=500
        )
        print(f"结果: Exact={results['exact']:.2f}, F1={results['f1']:.2f}, 测试样本数={results['total']}")

if __name__ == "__main__":
    main()


[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


加载训练数据...
加载验证数据...
拆分上下文为chunk...
拆分后上下文chunk数量: 32102
加载句向量模型...
GPU上计算所有chunk向量...


Embedding chunks: 100%|██████████| 126/126 [04:56<00:00,  2.35s/it]


构建BM25索引...
加载生成模型和tokenizer: google/flan-t5-xl


/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


...

Eval QAs top_n=4 beam=8 tokens=16 max_chars=5: 100%|██████████| 500/500 [04:57<00:00,  1.68it/s]


结果: Exact=52.20, F1=52.41, 测试样本数=500

===== 测试 max_context_chars = 10 =====


Eval QAs top_n=4 beam=8 tokens=16 max_chars=10: 100%|██████████| 500/500 [04:55<00:00,  1.69it/s]


结果: Exact=52.20, F1=52.41, 测试样本数=500

===== 测试 max_context_chars = 20 =====


Eval QAs top_n=4 beam=8 tokens=16 max_chars=20:   8%|▊         | 39/500 [00:20<03:33,  2.16it/s]

Eval QAs top_n=4 beam=8 tokens=16 max_chars=50[04:54<00:00,  1.70it/s]
结果: Exact=52.20, F1=52.41, 测试样本数=500

===== 测试 max_context_chars = 100 =====
Eval QAs top_n=4 beam=8 tokens=16 max_chars=100[04:54<00:00,  1.70it/s]
结果: Exact=52.20, F1=52.41, 测试样本数=500

# beam与tmp

In [1]:
import json
import collections
import re
import string
import numpy as np
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from sentence_transformers import SentenceTransformer
from tqdm import tqdm
from rank_bm25 import BM25Okapi
import nltk

nltk.download('punkt')
from nltk.tokenize import sent_tokenize

# --------- SQuAD Evaluation Functions ---------
def normalize_answer(s):
    if s is None:
        return ""
    def remove_articles(text):
        return re.sub(r'\b(a|an|the)\b', ' ', text)
    def white_space_fix(text):
        return ' '.join(text.split())
    def remove_punc(text):
        exclude = set(string.punctuation)
        return ''.join(ch for ch in text if ch not in exclude)
    def lower(text):
        return text.lower()
    return white_space_fix(remove_articles(remove_punc(lower(s))))

def get_tokens(s):
    if not s:
        return []
    return normalize_answer(s).split()

def compute_exact(a_gold, a_pred):
    return int(normalize_answer(a_gold) == normalize_answer(a_pred))

def compute_f1(a_gold, a_pred):
    gold_toks = get_tokens(a_gold)
    pred_toks = get_tokens(a_pred)
    common = collections.Counter(gold_toks) & collections.Counter(pred_toks)
    num_same = sum(common.values())
    if len(gold_toks) == 0 or len(pred_toks) == 0:
        return int(gold_toks == pred_toks)
    if num_same == 0:
        return 0
    precision = 1.0 * num_same / len(pred_toks)
    recall = 1.0 * num_same / len(gold_toks)
    return 2 * precision * recall / (precision + recall)

def get_raw_scores(dataset, preds):
    exact_scores = {}
    f1_scores = {}
    for article in dataset:
        for p in article['paragraphs']:
            for qa in p['qas']:
                qid = qa['id']
                gold_answers = [a['text'] for a in qa.get('answers', []) if normalize_answer(a.get('text', ''))]
                if not gold_answers:
                    gold_answers = ['']
                a_pred = preds.get(qid, "")
                exact_scores[qid] = max(compute_exact(a, a_pred) for a in gold_answers)
                f1_scores[qid] = max(compute_f1(a, a_pred) for a in gold_answers)
    return exact_scores, f1_scores

def make_eval_dict(exact_scores, f1_scores):
    total = len(exact_scores)
    return collections.OrderedDict([
        ('exact', 100.0 * sum(exact_scores.values()) / total if total else 0),
        ('f1', 100.0 * sum(f1_scores.values()) / total if total else 0),
        ('total', total),
    ])

# --------- 上下文拆分和检索 ---------
def chunk_context(text, max_tokens=100):
    sentences = sent_tokenize(text)
    chunks = []
    current_chunk = []
    current_len = 0
    for sent in sentences:
        sent_len = len(sent.split())
        if current_len + sent_len > max_tokens and current_chunk:
            chunks.append(' '.join(current_chunk))
            current_chunk = [sent]
            current_len = sent_len
        else:
            current_chunk.append(sent)
            current_len += sent_len
    if current_chunk:
        chunks.append(' '.join(current_chunk))
    return chunks

def build_bm25_index(context_chunks):
    tokenized_corpus = [ctx.split() for ctx in context_chunks]
    bm25 = BM25Okapi(tokenized_corpus)
    return bm25

def hybrid_search(question, bm25, embedder, context_chunks, all_ctx_emb,
                  k=1, bm25_top_n=5, sim_threshold=0.3):
    tokenized_query = question.split()
    bm25_scores = bm25.get_scores(tokenized_query)
    bm25_top_idx = np.argsort(bm25_scores)[::-1][:bm25_top_n]
    candidate_contexts = [context_chunks[i] for i in bm25_top_idx]
    candidate_embs = all_ctx_emb[bm25_top_idx]

    q_emb = embedder.encode([question], convert_to_numpy=True, device='cuda', normalize_embeddings=True, show_progress_bar=False)
    q_emb = np.asarray(q_emb, dtype=np.float32)

    sims = np.dot(candidate_embs, q_emb.T).squeeze(1)

    filtered_idxs = np.where(sims >= sim_threshold)[0]
    if len(filtered_idxs) == 0:
        filtered_idxs = np.arange(len(sims))

    filtered_sims = sims[filtered_idxs]
    filtered_bm25_idx = bm25_top_idx[filtered_idxs]

    bm25_scores_filtered = np.array(bm25_scores)[filtered_bm25_idx]

    bm25_norm = (bm25_scores_filtered - bm25_scores_filtered.min()) / (bm25_scores_filtered.max() - bm25_scores_filtered.min() + 1e-8)
    sim_norm = (filtered_sims - filtered_sims.min()) / (filtered_sims.max() - filtered_sims.min() + 1e-8)

    combined_scores = 0.5 * bm25_norm + 0.5 * sim_norm

    topk_idx = np.argsort(combined_scores)[::-1][:k]
    selected_contexts = [context_chunks[filtered_bm25_idx[i]] for i in topk_idx]
    return selected_contexts

def generate_answer(question, contexts, tokenizer, model, max_new_tokens=16, num_beams=1, temperature=0.0, max_context_chars=1500):
    device = next(model.parameters()).device
    total_chars = 0
    limited_contexts = []
    for c in contexts:
        if total_chars + len(c) > max_context_chars:
            break
        limited_contexts.append(c)
        total_chars += len(c)

    context_text = "\n".join(limited_contexts)
    prompt = f"Question: {question}\nContext:\n{context_text}\nAnswer (if no answer, respond with 'no answer'):"

    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=512)
    inputs = {k: v.to(device) for k, v in inputs.items()}

    outputs = model.generate(
        **inputs,
        max_new_tokens=max_new_tokens,
        num_beams=num_beams,
        temperature=temperature,
        do_sample=(temperature > 0),
        no_repeat_ngram_size=3,
        early_stopping=True
    )
    answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
    if answer.startswith(prompt):
        answer = answer[len(prompt):]
    answer = answer.strip()
    if answer.lower() in ["false", "no answer", "none", ""]:
        answer = ""
    return answer

def load_squad_dataset(path):
    with open(path, 'r', encoding='utf-8') as f:
        data_json = json.load(f)
    return data_json['data']

def run_evaluation(dev_data, bm25, embedder, context_chunks, all_ctx_emb, tokenizer, model,
                   k=1, sim_threshold=0.3,
                   bm25_top_n=4, num_beams=4, temperature=0.0,
                   max_new_tokens=16,
                   max_context_chars=1500,
                   max_test=500):

    preds = {}
    count = 0
    total_qas = sum(len(para["qas"]) for article in dev_data for para in article["paragraphs"])
    max_test = min(max_test, total_qas)

    with tqdm(total=max_test, desc=f"Eval QAs beam={num_beams} temp={temperature}") as pbar:
        for article in dev_data:
            for para in article["paragraphs"]:
                for qa in para["qas"]:
                    if count >= max_test:
                        break
                    qid = qa["id"]
                    question = qa["question"]

                    retrieved_contexts = hybrid_search(
                        question, bm25, embedder, context_chunks, all_ctx_emb,
                        k=k, bm25_top_n=bm25_top_n, sim_threshold=sim_threshold)
                    answer = generate_answer(
                        question, retrieved_contexts, tokenizer, model,
                        max_new_tokens=max_new_tokens, num_beams=num_beams,
                        temperature=temperature,
                        max_context_chars=max_context_chars)
                    preds[qid] = answer

                    count += 1
                    pbar.update(1)
                if count >= max_test:
                    break
            if count >= max_test:
                break

    filtered_dev_data = []
    for article in dev_data:
        new_article = {"paragraphs": []}
        for para in article["paragraphs"]:
            new_para = {"qas": [], "context": para["context"]}
            for qa in para["qas"]:
                if qa["id"] in preds:
                    new_para["qas"].append(qa)
            if new_para["qas"]:
                new_article["paragraphs"].append(new_para)
        if new_article["paragraphs"]:
            filtered_dev_data.append(new_article)

    exact_scores, f1_scores = get_raw_scores(filtered_dev_data, preds)
    eval_results = make_eval_dict(exact_scores, f1_scores)
    return eval_results

def main():
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    train_file = '/kaggle/input/squad-train/train-v2.0.json'
    dev_file = '/kaggle/input/squad-dev/dev-v2.0.json'

    sentence_transformer_name = 'sentence-transformers/all-mpnet-base-v2'
    model_name = "google/flan-t5-xl"

    print("加载训练数据...")
    train_data = load_squad_dataset(train_file)
    print("加载验证数据...")
    dev_data = load_squad_dataset(dev_file)

    print("拆分上下文为chunk...")
    context_chunks = []
    for article in train_data:
        for para in article["paragraphs"]:
            chunks = chunk_context(para["context"], max_tokens=100)
            context_chunks.extend(chunks)
    print(f"拆分后上下文chunk数量: {len(context_chunks)}")

    print("加载句向量模型...")
    embedder = SentenceTransformer(sentence_transformer_name, device=device)

    print("GPU上计算所有chunk向量...")
    batch_size_embed = 256
    all_ctx_emb = []
    for i in tqdm(range(0, len(context_chunks), batch_size_embed), desc="Embedding chunks"):
        batch = context_chunks[i:i+batch_size_embed]
        batch_emb = embedder.encode(batch, convert_to_numpy=True, device=device, normalize_embeddings=True, show_progress_bar=False)
        all_ctx_emb.append(batch_emb)
    all_ctx_emb = np.vstack(all_ctx_emb)

    print("构建BM25索引...")
    bm25 = build_bm25_index(context_chunks)

    print(f"加载生成模型和tokenizer: {model_name}")
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSeq2SeqLM.from_pretrained(model_name).to(device)
    model.eval()

    beam_sizes = [16, 20, 25]
    temperatures = [0.0, 0.3, 0.7, 1.0]

    for beam in beam_sizes:
        for temp in temperatures:
            print(f"\n===== 测试 beam={beam}, temperature={temp} =====")
            results = run_evaluation(
                dev_data, bm25, embedder, context_chunks, all_ctx_emb, tokenizer, model,
                k=1,
                sim_threshold=0.3,
                bm25_top_n=4,
                num_beams=beam,
                temperature=temp,
                max_new_tokens=16,
                max_context_chars=10,
                max_test=500
            )
            print(f"结果: Exact={results['exact']:.2f}, F1={results['f1']:.2f}, 测试样本数={results['total']}")

if __name__ == "__main__":
    main()


[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


加载训练数据...
加载验证数据...
拆分上下文为chunk...
拆分后上下文chunk数量: 32102
加载句向量模型...
GPU上计算所有chunk向量...


Embedding chunks: 100%|██████████| 126/126 [04:45<00:00,  2.27s/it]


构建BM25索引...
加载生成模型和tokenizer: google/flan-t5-xl


/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


...

结果: Exact=52.20, F1=52.33, 测试样本数=500

===== 测试 beam=20, temperature=0.0 =====


Eval QAs beam=20 temp=0.0: 100%|██████████| 500/500 [05:35<00:00,  1.49it/s]


结果: Exact=52.20, F1=52.40, 测试样本数=500

===== 测试 beam=20, temperature=0.3 =====


Eval QAs beam=20 temp=0.3:   5%|▌         | 27/500 [00:12<03:41,  2.14it/s]


KeyboardInterrupt: 